# **Sound Pressure Level Processing**

This notebook was developed to post process SPL data collected in the [Soundscape Project](https://www.italy-croatia.eu/web/soundscape).
Within the Soundscape Project, underwater noise was recorded in the North Adriatic See.    
A monitoring network of nine stations was set up in February 2020.                     
<img src="pics/Stations.png" width="40%">       

The raw wav data were processed using the web app [Audio Noise Processing](https://anp.soundscape.ve.ismar.cnr.it/), <span style='color:blue'> ANP</span>, developed within the Soundscape project, and and 1, 20 and 60 sec averaged Sound Pressure Level data, <span style='color:blue'> SPL</span>, were calculated for different 1/3 octave bands.  

This notebook is based on Python 3 kernel.      
When the user is asked to put an input, **<span style='color:red'>it is marked in red</span>**.

Input data are supposed to be the hdf5 file according to ICES (International Council for the Exploration of the Sea) continuous noise data portal specification (https://www.ices.dk/data/data-portals/Pages/Continuous-Noise.aspx - last read in 2022/06).   
For each station, hdf5 files are supposed to be in the same folder.
User will be asked to put the relative path of input and output directories.        

Table and graphs are produced using these statistical measures starting from SPLs data: 

**Exceedance Level ($L_{XX}$):** <span style='color:green'>the SPL value that is exceeded by the XX% of the time in a period (median is the $L_{50}$ level)</span>

**Averaged SPL:** <span style='color:green'>$10log_{10}(\frac{1}{n}\sum\limits_{i=1} ^{n} 10^{\frac{SPL_{i}}{10}})$ for each $SPL_{i}$ in a period for a given frequency</span>

**Total SPL:** <span style='color:green'>$10log_{10}(\sum\limits_{i=1} ^{n} 10^{\frac{SPL_{i}}{10}})$ for each $SPL_{i}$ in a period for a given frequency</span>

**Averaged SPL of different frequencies:** <span style='color:green'>$10log_{10}(\frac{1}{n}\sum\limits_{f=1} ^{n} 10^{\frac{SPL_{f,t}}{10}})$ for each $SPL_{f}$ of different 1/3 octave band frequencies at a time t</span>

**Total SPL of different frequencies:** <span style='color:green'>$10log_{10}(\sum\limits_{f=i} ^{n} 10^{\frac{SPL_{f,t}}{10}})$ for each $SPL_{f}$ of different 1/3 octave band frequencies at a time t</span>

**<span style='color:green'>Antonio Petrizzo, Picciulin Marta, Michol Ghezzo, Fantina Madricardo                 
    CNR-ISMAR                  
    antonio.petrizzo@cnr.it                       
    version 12/22       
</span>**

## **Necessary code**

**<span style='color:green'>Importing necessary modules/functions.         
Run next cells once when opening the notebook</span>**

In [ ]:
#check for packages
import sys
import subprocess
import pkg_resources

required = {'matplotlib','numpy','pandas','seaborn','scipy','openpyxl','statsmodels','h5py'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    print('install')
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)
#end check
    
%matplotlib inline
import csv
from datetime import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import math as mat
import seaborn as sns
import warnings
from matplotlib.dates import MonthLocator
from matplotlib.dates import WeekdayLocator
from matplotlib.dates import DayLocator
from matplotlib.dates import HourLocator
from matplotlib.dates import MinuteLocator
import time
from scipy import stats
import scipy
import openpyxl
import h5py

import seaborn.categorical
seaborn.categorical._Old_Violin = seaborn.categorical._ViolinPlotter
from statsmodels.stats.diagnostic import lilliefors

class _My_ViolinPlotter(seaborn.categorical._Old_Violin):

    def __init__(self, *args, **kwargs):
        super(_My_ViolinPlotter, self).__init__(*args, **kwargs)
        self.gray='gray'

seaborn.categorical._ViolinPlotter = _My_ViolinPlotter

dayInSec = 60*60*24

#Defining some functions


def median(df,column, span):  # mediana calcolata su base oraria, giornaliera o minuto
    #median over a period defined by span: 1H, 1M, 1D for hour, minute, day
    means = df.groupby(pd.Grouper(freq=span)).median()
    va = means[column].values
    da = means[column].index

    return da, va
    
def perc(df,column, span,q1,q2):  # q1 e q2 percentile (es 0.25 e 0.75) su base oraria, giornaliera o minuto
    #q1 and q2 percentiles over a period defined by span: 1H, 1M, 1D for hour, minute, day
    percQ1 = df.groupby(pd.Grouper(freq=span)).quantile(q1)
    percQ2 = df.groupby(pd.Grouper(freq=span)).quantile(q2)
    vaQ1 = percQ1[column].values
    vaQ2 = percQ2[column].values
    da = percQ1[column].index

    return da, vaQ1, vaQ2

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

def totalSPL(spl,f):
    #calculate total and averaged SPL
    #input spl: pandas dataframe
    #      f: frequency, int
    #if f == -9999:
        #f = 'FT'
        #sp = spl[f]
    #else:
        #sp = spl[F[f]]
    sp = spl[F[f]]   
    SUM = 0
    n = 0
    for s in sp.values:
        if np.isnan(s):#to avoid nan issue
            continue
        n += 1
        SUM = SUM + 10**(s/10.)
        
    if n == 0:
        SPLT = np.nan
        SPL_av = np.nan
    else:
        SPLT = 10*np.log10(SUM)
        SPL_av = 10*np.log10(SUM/n)
    
    return SPLT, SPL_av
    #print(FREQ[f] + ' SPL_tot = ', SPLT)
    #print(FREQ[f] + ' SPL_av = ', SPL_av)
    
def totalXperiod(df,f, span):  
    #Total and averaged SPL over a period defined by span: 1H, 1M, 1D for hour, minute, day
    total = []
    aver = []
    ddd = df.groupby(pd.Grouper(freq=span)).apply(totalSPL, f=f)
    da = ddd.index
    for l in ddd.values: 
        total.append(l[0])
        aver.append(l[1])
    
    
    return da, total, aver

def sumFreq(row,freq):
    #total power in different frequencies
    n = 0
    SUM = 0
    
    for fr in freq:
        s = row[F[fr]]
        if np.isnan(s):#to avoid nan issue
            continue
        n += 1
        SUM = SUM + 10**(s/10.)
    if n == 0:
        ftot = np.nan
        fmean = np.nan
    else:
        ftot = 10*np.log10(SUM)
        fmean = 10*np.log10(SUM/n)
    return ftot ,fmean

<span style='color:red'>*________________________________________________________________*                        
</span>
<span style='color:Red'>**Defining some inputs.**</span>    

In [ ]:
#label of graph
lab = 'SPL [dB re 1 \u03BCPa]' # graph label

# general extension of output files. it can be changed later when selecting the time window
ext = '_tot' 

#Defining Dictionaries

# name of stations
STATIONS = {1:'MS1_acqua_alta',2:'MS2_Azalea',3:'MS3_Ancona',4:'MS4_PALOMA',5:'MS5_Susak',6:'MS6_LOSINJ',7:'MS7_Zirje',8:'MS8_SPLIT',
            9:'MS9_IVANA'}

# short name of stations. They are also the name of hdf5 input files
STATIONS_SHORT = {1:'MS1',2:'MS2',3:'MS3',4:'MS4',5:'MS5',6:'MS6',7:'MS7',8:'MS8',9:'MS9'}

#Next values should not be changed unless input  files does not change structure
######
FREQ = {0:'20-10000Hz',5:'5Hz',10:'10Hz',12.5:'12.5Hz',16:'16Hz',20:'20Hz',25:'25Hz',31.5:'31.5Hz',40:'40Hz',50:'50Hz',63:'63Hz',80:'80Hz',
        100:'100Hz',125:'125Hz',160:'160Hz',200:'200Hz',250:'250Hz',315:'315Hz',400:'400Hz',500:'500Hz',
        630:'630Hz',800:'800Hz',1000:'1000Hz',1250:'1250Hz',1600:'1600Hz',2000:'2000Hz',2500:'2500Hz',3150:'3150Hz',
        4000:'4000Hz',5000:'5000Hz',6300:'6300Hz',8000:'8000Hz',10000:'10000Hz',12500:'12500Hz',16000:'16000Hz',20000:'20000Hz'}

F = {0:'20-10000',5:'5_1/3_octave',10:'10_1/3_octave',12.5:'12.5_1/3_octave',16:'16_1/3_octave',20:'20_1/3_octave',25:'25_1/3_octave',
     31.5:'31.5_1/3_octave',40:'40_1/3_octave',50:'50_1/3_octave',63:'63_1/3_octave',80:'80_1/3_octave',100:'100_1/3_octave',125:'125_1/3_octave',
     160:'160_1/3_octave',200:'200_1/3_octave',250:'250_1/3_octave',315:'315_1/3_octave',400:'400_1/3_octave',500:'500_1/3_octave',630:'630_1/3_octave',
     800:'800_1/3_octave',1000:'1000_1/3_octave',1250:'1250_1/3_octave',1600:'1600_1/3_octave',2000:'2000_1/3_octave',2500:'2500_1/3_octave',
     3150:'3150_1/3_octave',4000:'4000_1/3_octave',5000:'5000_1/3_octave',6300:'6300_1/3_octave',8000:'8000_1/3_octave',
     10000:'10000_1/3_octave',12500:'12500_1/3_octave',16000:'16000_1/3_octave',20000:'20000_1/3_octave'}

col = ['10_1/3_octave','12.5_1/3_octave','16_1/3_octave','20_1/3_octave','25_1/3_octave',
     '31.5_1/3_octave','40_1/3_octave','50_1/3_octave','63_1/3_octave','80_1/3_octave','100_1/3_octave','125_1/3_octave',
     '160_1/3_octave','200_1/3_octave','250_1/3_octave','315_1/3_octave','400_1/3_octave','500_1/3_octave','630_1/3_octave',
     '800_1/3_octave','1000_1/3_octave','1250_1/3_octave','1600_1/3_octave','2000_1/3_octave','2500_1/3_octave',
     '3150_1/3_octave','4000_1/3_octave','5000_1/3_octave','6300_1/3_octave','8000_1/3_octave',
     '10000_1/3_octave','12500_1/3_octave','16000_1/3_octave','20000_1/3_octave']


<span style='color:red'>*________________________________________________________________*                        
</span>
<span style='color:red'>**Defining input/output directories**                      </span>

to set the path of input folders as 'd:/soundscape/input/' use:
```python
dirInput = 'd:/soundscape/input/'
```    
    
to set the output folder as 'd:/soundscape/output/' use:
```python
dirOutput = 'd:/soundscape/output/'
``` 
if the output folder doesn't exist, it will be created.    

In [ ]:
dirInput = '20s/' 
dirOutput = 'output/'

if os.path.isdir(dirOutput) == False:
    os.mkdir(dirOutput)

<span style='color:red'>*________________________________________________________________*                        
</span>
<span style='color:Red'>**Select the stations to be read.**                         
</span>
From 1 (MS1) to 9 (MS9)      
i.g. to load data of stations MS1, MS2 and MS3 set:</span>
```python
stations = [1,2,3]
```
to load data of stations MS1 set:
```python
stations = [1]
```

In [ ]:
stations = [1,2,3,4,5,6,7,8,9]

 **<span style='color:green'>Read the data</span>**        
 Stations name are set in STATIONS_SHORT dictionary                  
 ***it may take a while***

In [ ]:
t = time.time()

list_df = []

n_sta =len(STATIONS)

first_data = [None] * (n_sta+1)
last_data = [None] * (n_sta+1)

for st in stations:
    myfile = STATIONS_SHORT[st] + '.hdf5'
    f = h5py.File(dirInput + myfile,'r+')
    print(myfile)
    ct = f['Metadata']['AveragingTime']
    sec = ct[0]
    print('averaging time (sec):',sec)
    fr = f['Metadata']['FrequencyIndex']
    print('freq:')
    for fi in fr:
        print('{:9.1f}'.format(fi)) 
   
    #arrange date
    mydate_time = []
    for stri in f['Data']['DateTime']:
        stri = str(stri)
        
        try:
            stri = stri.replace("b'",'')#("b'",'')
            stri = stri.replace("Z'",'')#("Z'",'')

            stri = stri.replace("T",' ')#("T",' ')
            mydate_time.append(datetime.strptime(stri, "%Y-%m-%d %H:%M:%S")) 
        except:
            stri = stri.replace("b'",'')#("b'",'')
            stri = stri.replace("Z",'')#("Z'",'')

            stri = stri.replace("T",' ')#("T",' ')
            mydate_time.append(datetime.strptime(stri, "%Y-%m-%d %H:%M:%S")) 
            
    #arrange data
    mydata = []
    for d in f['Data']['LeqMeasurementsOfChannel1']:
        mydata.append(d)

    merged_st_df = pd.DataFrame(mydata,columns = col)
    merged_st_df.index = mydate_time
    merged_st_df.rename_axis('datetime', inplace=True)
    
    merged_st_df['Station'] = STATIONS_SHORT[st]
    
    merged_st_df = merged_st_df.sort_index()
    
    list_df.append(merged_st_df)
    
    ddf = merged_st_df.loc[merged_st_df['Station'] == STATIONS_SHORT[st]]
    
    first_data[st] = ddf.index[0]
    last_data[st] = ddf.index[-1]
    
    print(STATIONS[st])
    print('First SPL data at: ', first_data[st])
    print('Last  SPL data at: ', last_data[st])
        
    delta = (last_data[st] - first_data[st])
    delta_sec = delta.total_seconds()
    n_exp = int(delta_sec/sec)+1
    percOfData = len(ddf)/n_exp
    
    print('In the selected range:')
    print('Expected data: {:9}'.format(n_exp))
    print('Total data:    {:9}'.format(len(ddf)))
    print('Percent of data: {:2.2%}'.format(percOfData))
   
    print('_________________________')

df_st_tot = []

for d in list_df:
    df_st_tot.append(d)

merged_box = pd.concat(df_st_tot)

first_time = first_data.copy()
last_time = last_data.copy()

elapsed = time.time() - t

    
print('\n')
print('execution time (s)', elapsed)

## **End of necessary code**

## **Data Processing**

<a id =20> </a>
<span style='color:black'>Once loaded the data, the most time comsuming task, different graphs/tables can be outputed.   
Date range and frequency can be easily modified</span>

<span style='color:red'>*________________________________________________________________*                        
</span><span style='color:Red'>**Define a time window setting an initial and final time.**</span>                        
Use this format:   
datetime(year, month, day, hour= , minute= , second= )   
i.g. to use data between 2020-06-01 00:00:00 and 2020-08-31 23:59:59 set:</span>    
```python
t_init = datetime(2020, 6, 1, hour=0, minute=0, second=0)
t_final = datetime(2020, 8, 31, hour=23, minute=59, second=59)
```    
An extension to add to the output file name can be defined.                      
```python
_ext = '_myext'
```   

In [ ]:
t_init = datetime(2020, 3, 1, hour=0, minute=0, second=0)#t_init = datetime(2020, 4, 1, hour=0, minute=0, second=0)
t_final = datetime(2021, 6, 30, hour=23, minute=59, second=59)#t_final = datetime(2021, 3, 31, hour=23, minute=59, second=40)
ext = '_20sec_check'

**<span style='color:green'>Filtering data in the time window</span>**

In [ ]:
df_st_tot = []
#merged_box = []
for d in list_df:
    df_st_tot.append(d.loc[t_init:t_final])

merged_box = pd.concat(df_st_tot)


for st in stations:
    first_time[st] = t_init
    last_time[st] = t_final
    
    df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
    if len(df)>0:
        print(STATIONS_SHORT[st] + '...done   ' + str(first_time[st]) + '--' + str(last_time[st]))
    else:
        print('station', STATIONS_SHORT[st], 'has no data') 

[Go to graph list](#0) or create some tables

<span style='color:Green'>**Run next 2 cells if you want to calculate total and average SPL over different frequencies for each timestep**                           
</span>
<span style='color:red'>*________________________________________________________________*                        
</span><span style='color:Red'>**i.g. to calculate SPL in 63,80,100,125 frequencies set:**   
```python
freq = [63,80,100,125]
f_id = -9999 #numeric
f_name = '63-2000'
f_id_mean = -8889 #numeric
f_name_mean = '63-2000_Mean'
```    
where f_id is the frequency code to use in next operations and f_name its label

In [ ]:
freq = [10,12.5,16,20,25,31.5,40,50,63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000]
f_id = -9999
f_name = '63-2000_Tot'
f_id_mean = -9998 
f_name_mean = '63-2000_Mean'

In [ ]:
t = time.time()
try:
    F.pop(f_id)
    FREQ.pop(f_id)
    F.pop(f_id_mean)
    FREQ.pop(f_id_mean)
    merged_box = merged_box.drop([f_name], axis=1,errors='ignore')
    merged_box = merged_box.drop([f_name_mean], axis=1,errors='ignore')
    F[f_id] = f_name
    FREQ[f_id] = f_name
    F[f_id_mean] = f_name_mean
    FREQ[f_id_mean] = f_name_mean
    
except:
    F[f_id] = f_name
    FREQ[f_id] = f_name
    F[f_id_mean] = f_name_mean
    FREQ[f_id_mean] = f_name_mean
    
merged_box[[f_name, f_name_mean]] = merged_box.apply(lambda row: sumFreq(row,freq), axis=1,result_type = 'expand')

elapsed = time.time() - t

    
print('\n')
print('execution time (s)', elapsed)

[Go to graph list](#0) or create some tables

## **Tables**

<a id =T1> </a> **Data values along a temporal step**        
For each station and frequency selected, it calculates a value for each temporal step (eg. hourly or daily median/average/Excedance Levels).     
Output will be put in dirOutput/tab folder.                           
Set span = '20s' (20 seconds) to extract raw spl data. 

<a id =30> </a>
<span style='color:red'>*________________________________________________________________*                        
</span>Select stations, frequencies and time span:
```python
CalculateMedian = True #True/False for calculating medians
CalculateAverage = True #True/False for calculating average SPL
CalculateTotal = False #True/False for calculating total SPL
CalculateEL = [10,50,90] #define the Excedance level. leave blank [] to skip it

sts = [4,6] #define the stations to use. Leave blank to use the loaded stations 
freq = [0,5,10,12.5,16,20,25,31.5,40,50,63,80,100,125,160,200,250,315,400,
        500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,1000,12500,16000,20000] #define the frequencies to use. Leave blank to use the 63,125,250,4000 Hz 
span = '1H' (s: second, min: minute; H: hour; D: day: W: week; M: month; Y:year) # define the temporal step. Leave blank for 1H
```

In [ ]:
CalculateMedian = True
CalculateAverage = True
CalculateTotal = False
CalculateEL = []
sts = [1,2]
freq = [10,12.5,16,20,25,31.5,40,50,63,80,100,125,160,200,250,315,400,
        500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,1000,12500,16000,20000,-9999,-9998]
span = '60s'

In [ ]:
if os.path.isdir(dirOutput + 'tab') == False:
    os.mkdir(dirOutput + 'tab')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals():
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]

if 'span' in locals():
    if len(span) == 0:
        span = '1H'
else:
    span = '1H'

    
for st in sts:
    if st in stations:
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        df_CSV = pd.DataFrame()
        df1_CSV = pd.DataFrame()
        df2_CSV = pd.DataFrame()
        df_perc = []  
        if len(CalculateEL) > 0:
            for p in CalculateEL:
                df_perc.append(pd.DataFrame())
                
        for f in freq:
            if CalculateMedian:
                da , va = median(df,F[f], span)
                df_CSV[F[f]] = va
            if CalculateAverage or CalculateTotal:
                da1,tot1,av1 = totalXperiod(df,f, span)
                if CalculateAverage:
                    df1_CSV[F[f]] = av1
                if CalculateTotal:
                    df2_CSV[F[f]] = tot1
            if len(CalculateEL) > 0:
                c = 0
                for p in CalculateEL:
                    (da2,Yq1,Yq2) = perc(df,F[f],span,(100-p)/100.,(100-p)/100.)
                    df_perc[c][F[f]] = Yq1
                    c += 1
        

        if CalculateMedian:
            df_CSV.index = da
            df_CSV.to_csv(dirOutput + 'tab/' + STATIONS_SHORT[st] + "_median" + ext + '_' + span + '.csv', compression=None)
            corrmat = df_CSV.corr()
            #print(corrmat)
            #corrmat.to_csv(dirOutput + 'tab/' + STATIONS_SHORT[st] + "corr" + ext + '_' + span + '.csv', compression=None)
        if CalculateAverage:
            df1_CSV.index = da1
            df1_CSV.to_csv(dirOutput + "tab/" + STATIONS_SHORT[st] + "_aver" + ext + '_' + span + '.csv', compression=None)
        if CalculateTotal:
            df2_CSV.index = da1
            df2_CSV.to_csv(dirOutput + "tab/" + STATIONS_SHORT[st] + "_total" + ext + '_' + span + '.csv', compression=None)
        if len(CalculateEL) > 0:
            c = 0
            for p in CalculateEL:
                df_perc[c].index = da2
                df_perc[c].to_csv(dirOutput + "tab/" + STATIONS_SHORT[st] + "_EL" + str(p) + ext + '_' + span + '.csv', compression=None)            
                c += 1               
            
        print('_____________________________')
        print(STATIONS[st] + '... done')
        if 'df' in locals():
            del df
        if 'df_CSV' in locals():
            del df_CSV
        if 'df1_CSV' in locals():
            del df1_CSV
        if 'df2_CSV' in locals():
            del df2_CSV
        if 'df3_CSV' in locals():
            del df3_CSV
        if 'df_perc' in locals():
            del df_perc
        
del sts
del freq
del span 

<a id =T2> </a> **Statistics**        
For each station and frequency selected, it calculates some statistics.            
Output will be put in dirOutput/tab folder.

<span style='color:red'>*________________________________________________________________*                        
</span>Select stations (leave it empty to use all loaded stations) and frequencies (Leave blank to use the 63,125,250,4000 Hz)

In [ ]:
sts = [1,2]
freq = [10,12.5,16,20,25,31.5,40,50,63,80,100,125,160,200,250,315,400,
        500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,1000,12500,16000,20000]

In [ ]:
if os.path.isdir(dirOutput + 'tab') == False:
    os.mkdir(dirOutput + 'tab')


if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
        
for st in sts:
    if st in stations:
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        list_fre = []
        list_mean = []
        list_50 = []
        list_min = []
        list_max = []
        list_75 = []
        list_25 = []
        list_90 = []
        list_10 = []
        list_95 = []
        list_5 = []
        list_param = []
        list_TSPL = []
        list_ASPL = []
        
        delta = (last_time[st] - first_time[st])
        delta_sec = delta.total_seconds()
        n_exp = int(delta_sec/sec)+1
        percOfData = len(df)/n_exp
            
        
        if len(df)>0:
            df_CSV = pd.DataFrame()
            df_CSV2 = pd.DataFrame()
            
            
            list_param.append('Station = ' + STATIONS[st])
            list_param.append('File = ' + str(sec) + 's')
            list_param.append('Start time = ' + str(first_time[st]))
            list_param.append('Last time = ' + str(last_time[st]))
            list_param.append('First data = ' + str(df.index[0]))
            list_param.append('Last data = ' + str(df.index[-1]))
            list_param.append('Expected data = {:9}'.format(n_exp))
            list_param.append('Total data = {:9}'.format(len(df)))
            list_param.append('Percent of data = {:2.2%}'.format(percOfData))
            


            for f in freq:
                
                
                list_fre.append(F[f])
                list_mean.append(df[F[f]].mean())
                list_50.append(df[F[f]].quantile(0.5))
                list_min.append(df[F[f]].quantile(0))
                list_max.append(df[F[f]].quantile(1))
                list_75.append(df[F[f]].quantile(0.25))
                list_25.append(df[F[f]].quantile(0.75))
                list_90.append(df[F[f]].quantile(0.1))
                list_10.append(df[F[f]].quantile(0.9))
                list_95.append(df[F[f]].quantile(0.05))
                list_5.append(df[F[f]].quantile(0.95))
                SPLT,SPL_av = totalSPL(df,f)
                list_TSPL.append(SPLT)
                list_ASPL.append(SPL_av)
                
                
           
        else:

            df_CSV = pd.DataFrame()
            df_CSV2 = pd.DataFrame()
            #df_CSV.index = freq
            list_param.append('Station = ' + STATIONS[st])
            list_param.append('File = ' + str(sec) + 's')
            list_param.append('Start time = ' + str(first_time[st]))
            list_param.append('Last time = ' + str(last_time[st]))
            list_param.append('First data = nan')
            list_param.append('Last data = nan')
            list_param.append('Expected data = {:9}'.format(n_exp))
            list_param.append('Total data = {:9}'.format(len(df)))
            list_param.append('Percent of data = {:2.2%}'.format(percOfData))
            for f in freq:
                list_fre.append(F[f])
                list_mean.append('nan')
                list_50.append('nan')
                list_min.append('nan')
                list_max.append('nan')
                list_75.append('nan')
                list_25.append('nan')
                list_90.append('nan')
                list_10.append('nan')
                list_95.append('nan')
                list_5.append('nan')
                list_TSPL.append('nan')
                list_ASPL.append('nan')
        
        df_CSV2['Stats'] = list_param
        df_CSV['Freq'] = list_fre                
        df_CSV['GeometricMean'] = list_mean
        df_CSV['50th percentile'] = list_50
        df_CSV['MIn'] = list_min
        df_CSV['Max']  = list_max
        df_CSV['75 EL'] = list_75
        df_CSV['25 EL'] = list_25
        df_CSV['90 EL'] = list_90
        df_CSV['10 EL'] = list_10
        df_CSV['95 EL'] = list_95
        df_CSV['5 EL'] = list_5
        df_CSV['Tot_SPL'] = list_TSPL
        df_CSV['Av_SPL'] = list_ASPL
        
        df_CSV3 = df_CSV2.append(df_CSV)
        
        df_CSV3.to_csv(dirOutput + "tab/" + STATIONS_SHORT[st] + "_stats" + ext + '.csv', compression=None)
    
        del df_CSV3
        del df_CSV2
        del df_CSV
        print('_____________________________')
        print(STATIONS_SHORT[st] + '...done')
               
    else:
        print('_____________________________')
        print('station', STATIONS[st], 'not loaded')
        
del sts
del freq

## **Graphs**

<span style='color:Red'>Select the graph.</span> 

<a id =0> </a>

| Graph | Example | Graph | Example |
|:-----:|:-------:|:-----:|:-------:|
|   [Spectrogram like](#G1)   |  <img src="pics/spectral.png" width="10%">    |   [Median Values of spectra](#G11)   |    <img src="pics/Frequecnies.png" width="20%">    |
|   [Plot of the data](#G2)   |  <img src="pics/plot.png" width="10%">    |   [Plot of daily median](#G3)    |    <img src="pics/daily.png" width="20%">    |
|   [KDE Density plot](#G4)   |    <img src="pics/density.png" width="10%">    |   [Histogram](#G5)   |    <img src="pics/histo.png" width="20%">    |
|   [Boxplot](#G6)   |    <img src="pics/Boxplot.png" width="10%">    |   [Violinplot](#G7)   |    <img src="pics/Violinplot.png" width="20%">    |
|   [Boxplot of day/night data](#G9)   |    <img src="pics/day_night.png" width="10%">    |   [Violinplot of day/night data](#G10)   |    <img src="pics/Violinplot_day_night.png" width="20%">
| [Plot of percentiles](#G8)   |    <img src="pics/percentile.png" width="10%">   |   [Percentiles comparison](#G12)   |    <img src="pics/percentiles_compare.png" width="20%"> 
|   [Boxplot of frequencies](#G13)   |    <img src="pics/Boxplot_cfr_freq.png" width="10%">    |   [Violinplot of frequencies](#G14)   |    <img src="pics/Violinplot_cfr_freq.png" width="20%">
|   [Boxplot of monthly data](#G15)   |    <img src="pics/Boxplot_monthly.png" width="10%">    |   [Violinplot of monthly data](#G16)   |    <img src="pics/Violinplot_monthly.png" width="20%">
|   [Boxplot of different periods](#G17)   |    <img src="pics/Boxplot_periods.png" width="10%">    |   [Violinplot of different periods](#G18)   |    <img src="pics/Violinplot_monthly.png" width="20%">     
|   [Plot of frequencies in different periods](#G19)   |    <img src="pics/Frequecnies_Periods.png" width="10%">    |               [Plot of Percentiles in different periods](#G180)   |    <img src="pics/percentiles_compare.png" width="20%">
|   [Plot of the percentiles for different stations](#G20)   |  <img src="pics/percentiles_compare_stations.png" width="10%">    |[Plot of the densities for different stations](#G23)   |  <img src="pics/densplotStaz.png" width="20%">    |
|   [Boxplot of stations](#G21)   |    <img src="pics/Boxplot_compare_stations.png" width="10%">    |   [Violinplot of stations](#G22)   |    <img src="pics/Violinplot_compare_stations.png" width="20%">    |

<a id =G1> </a> **Contour plot, spectrogram like**        
For each station selected, create a spectrogram like plot. If no frequenies are selected, it uses the whole band.     
Output will be put in dirOutput/contourPlot folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)             
Select first and last frequency.  (leave it empty to use 5 tot 20000 Hz)                
Select z lim, leave blank for auto.              
**input example:**
```python
sts = [1] #Stations to plot
freq = [5, 4000] #Frequencies range,[F1,F2] or [] for [5,20000]

zlim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
sts = [1,2]
freq = [10,4000]

zlim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'contourPlot') == False:
    os.mkdir(dirOutput + 'contourPlot')

lv = 30 #n levels

if not 'freq' in locals():
    freq = [5,20000]
if len(freq) == 0:
    F1 = 5
    F2 = 20000
    print('F: ', [F1,F2])
else:
    if freq[0] < 5:
        freq[0] = 5
    if freq[1] > 20000:
        freq[1] = 20000

    F1 = freq[0]
    F2 = freq[1]

    print('F: ', freq)

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:   
            a = (last_time[st] - first_time[st]).total_seconds()
           
            
            Y = []
            chk = df.loc[t_final:t_final]
            chk1 = df.loc[t_init:t_init]
            for key in FREQ:
                if key < F1 or key > F2:
                    continue
                Y.append(key)
            
                if len(chk) == 0:
                    df1 = pd.DataFrame([[np.nan]], columns=list([F[key]]), index=[t_final])
                    df= pd.concat([df, df1]) 
                if len(chk1) == 0:
                    df1 = pd.DataFrame([[np.nan]], columns=list([F[key]]), index=[t_init])
                    df= pd.concat([df, df1]) 
            
            df = df.sort_index()
            
            Z = df.loc[:, F[F1]:F[F2]].values
            X=df.index.values
            fig1, ax2 = plt.subplots(figsize=(11.7,8.3),constrained_layout=True) #(11.7/1.2,8.3/1.5)

            if not 'zlim' in locals() or len(zlim) == 0:
                CS = ax2.contourf(X, Y, Z.transpose(),levels = lv,cmap=plt.cm.Spectral_r)
            else:
                
                l1 = zlim[0] #Z lim inf
                l2 = zlim[1] #Z lim sup
                levels = np.linspace(l1, l2, lv)
                CS = ax2.contourf(X, Y, Z.transpose(),levels = levels,cmap=plt.cm.Spectral_r) 

            # Make a colorbar for the ContourSet returned by the contourf call.
            cbar = fig1.colorbar(CS)
            cbar.ax.set_ylabel('dB re 1 \u03BCPa', fontsize=15)

            plt.yscale('log')

            ax = plt.gca() #get axes
            

            if a > 60*dayInSec:
                ax.xaxis.set_major_locator(MonthLocator())
                formatter = mdates.DateFormatter('%m/%y') 
                ax.xaxis.set_major_formatter(formatter)
            elif a <= 60*dayInSec and a > 15*dayInSec:
                ax.xaxis.set_major_locator(DayLocator(interval=5))
                formatter = mdates.DateFormatter('%d/%m') 
                ax.xaxis.set_major_formatter(formatter)
            elif a <= 15*dayInSec and a > 1*dayInSec:
                ax.xaxis.set_major_locator(DayLocator())
                formatter = mdates.DateFormatter('%d/%m')
                ax.xaxis.set_major_formatter(formatter)
            elif a <= 1*dayInSec and a > .2*dayInSec:
                ax.xaxis.set_major_locator(HourLocator())
                formatter = mdates.DateFormatter('%H') 
                ax.xaxis.set_major_formatter(formatter)
            elif a <= 1*dayInSec and a > .2*dayInSec:
                ax.xaxis.set_major_locator(MinuteLocator(byminute=range(1,61,10)))
                formatter = mdates.DateFormatter('%H:%M') 
                ax.xaxis.set_major_formatter(formatter)
            else:
                ax.xaxis.set_major_locator(MinuteLocator(byminute=range(1,61,1)))
                formatter = mdates.DateFormatter('%M:%S') 
                ax.xaxis.set_major_formatter(formatter)


            

            plt.setp(ax.get_xticklabels(), rotation=0,fontsize=14)
            plt.setp(ax.get_yticklabels(), rotation=0,fontsize=14)
            plt.ylabel('Hz', fontsize=15)    

            
            xlim = [mdates.date2num(t_init),mdates.date2num(t_final)]
            plt.xlim = xlim
            plt.title(STATIONS_SHORT[st] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
            
            
            plt.savefig(dirOutput + 'contourPlot/' + STATIONS_SHORT[st] + "_" + str(F1) + "_" + str(F2) + "_spectral_day_" + str(sec) + "sec" + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            
        else:
            print('station', STATIONS[st], 'has no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
del sts

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G2> </a> **Plot of the data**           
Output will be put in dirOutput/plot folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select whether to plot daily median and total/average spl or not (Y or N, default N).                
Select stations (leave it empty to use all loaded stations)             
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
plot_median = 'n' #daily median
plot_total_spl = 'n' #daily total spl
plot_average_spl = 'n' #daily average spl

sts = [1] #Stations to plot
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
plot_median = 'n'
plot_total_spl = 'n'
plot_average_spl = 'n'

sts = []
freq = [63]

ylim = []
   

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'plot') == False:
    os.mkdir(dirOutput + 'plot')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]

    
if not 'plot_median' in locals():
    plot_median = 'n'
if not 'plot_total_spl' in locals():
    plot_total_spl = 'n'
if not 'plot_average_spl' in locals():
    plot_average_spl = 'n'

for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            print(last_time[st])
            print(first_time[st])
            a = (last_time[st] - first_time[st]).total_seconds()

            for f in freq:
                fig = plt.figure(figsize=(11.7,8.3)) 
                
                
                if a > 60*dayInSec:
                    print('more than 60 days!')
                    ax1 = plt.plot(df[F[f]].index,df[F[f]].values,'.',markersize = .5,ls ='',label=str(sec) + 'sec SPL')
                    ax = plt.gca() #get axes
                    ax.xaxis.set_major_locator(MonthLocator())
                    formatter = mdates.DateFormatter('%m/%y') 
                    ax.xaxis.set_major_formatter(formatter)
                elif a <= 60*dayInSec and a > 15*dayInSec:
                    print('between 15 and 60 days!')
                    ax1 = plt.plot(df[F[f]].index,df[F[f]].values,'.',markersize = .5,ls ='',label=str(sec) + 'sec SPL')
                    ax = plt.gca() #get axes
                    ax.xaxis.set_major_locator(DayLocator(interval=5))
                    formatter = mdates.DateFormatter('%d/%m') 
                    ax.xaxis.set_major_formatter(formatter)
                elif a <= 15*dayInSec and a > 1*dayInSec:
                    print('between 1 and 15 days!')
                    ax1 = plt.plot(df[F[f]].index,df[F[f]].values,'.',markersize = .5,ls ='',label=str(sec) + 'sec SPL')
                    ax = plt.gca() #get axes
                    ax.xaxis.set_major_locator(DayLocator())
                    formatter = mdates.DateFormatter('%d/%m') 
                    ax.xaxis.set_major_formatter(formatter)
                elif a <= 1*dayInSec and a > .2*dayInSec:
                    print('between 1/2 and 1 day!')
                    ax1 = plt.plot(df[F[f]].index,df[F[f]].values,'.',markersize = .5,ls ='',label=str(sec) + 'sec SPL')
                    ax = plt.gca() #get axes
                    ax.xaxis.set_major_locator(HourLocator())
                    formatter = mdates.DateFormatter('%H') 
                    ax.xaxis.set_major_formatter(formatter)
                elif a <= .2*dayInSec and a > 1/12*dayInSec:
                    print('between 1h and 1/2 day!')
                    ax1 = plt.plot(df[F[f]].index,df[F[f]].values,'.',markersize = 2,ls ='',label=str(sec) + 'sec SPL')
                    ax = plt.gca() #get axes
                    ax.xaxis.set_major_locator(MinuteLocator(byminute=range(1,61,10)))
                    formatter = mdates.DateFormatter('%H:%M') 
                    ax.xaxis.set_major_formatter(formatter)
                else:
                    print('less than 1 h!')
                    ax1 = plt.plot(df[F[f]].index,df[F[f]].values,'.',markersize = 12,ls ='',label=str(sec) + 'sec SPL')#markersize = 4
                    ax = plt.gca() #get axes
                    ax.xaxis.set_major_locator(MinuteLocator(byminute=range(1,61,1)))
                    formatter = mdates.DateFormatter('%H:%M') 
                    ax.xaxis.set_major_formatter(formatter)

                if plot_median == 'Y' or plot_median == 'y':
                    print('median')
                    timeSpan = '1D'
                    (da,va) = median(df,F[f],timeSpan)
                    ax1 = plt.plot(da+ pd.DateOffset(hours=12),va,'r-',markersize = 2, markerfacecolor='r', markeredgecolor='r',label='Daily Median')

                if plot_total_spl == 'Y' or plot_total_spl == 'y':
                    timeSpan = '1D'
                    da,totale,averaged = totalXperiod(df,f,timeSpan)
                    ax1 = plt.plot(da,totale,'k-o',markersize = 1, markerfacecolor='k', alpha=.5,markeredgecolor='k',label='Daily Total SPL')

                if plot_average_spl == 'Y' or plot_average_spl == 'y':
                    timeSpan = '1D'
                    da,totale,averaged = totalXperiod(df,f,timeSpan)
                    ax1 = plt.plot(da,averaged,'k-o',markersize = 1, markerfacecolor='k', alpha=1,markeredgecolor='k',label='Daily Averaged SPL')

                SPLT,SPL_av = totalSPL(df,f)
                if plot_total_spl == 'Y' or plot_total_spl == 'y':
                    la = 'Total SPL = ' + str('{:.2f}')
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10] + ' (' +  la.format(SPLT) + ')', fontsize=15,fontweight = 'bold')
                elif plot_average_spl == 'Y' or plot_average_spl == 'y':
                    la = 'Averaged SPL = ' + str('{:.2f}')
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10] + ' (' +  la.format(SPL_av) + ')', fontsize=15,fontweight = 'bold')
                else:
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=22,fontweight = 'bold')

                l10 = df[F[f]].quantile(0.9)
                l90 = df[F[f]].quantile(0.1)
                l50 = df[F[f]].quantile(0.5)
                #xlim = [df.index[0],df.index[-1]]
                xlim = [mdates.date2num(t_init),mdates.date2num(t_final)]

                plt.plot([xlim[0],xlim[1]],[l10,l10],'--k',linewidth=2)
                plt.plot([xlim[0],xlim[1]],[l90,l90],'--k',linewidth=2)
                plt.plot([xlim[0],xlim[1]],[l50,l50],'--k',linewidth=2,label = '10/50/90 EL')

                
                handles, labels = ax.get_legend_handles_labels()
                lgd = ax.legend(handles, labels, loc=1, bbox_to_anchor=(1.27, 1), fontsize=15)
                
                if 'ylim' in locals():
                    if len(ylim) > 0:
                        plt.ylim(ylim)
                        
              
                
                plt.ylabel(lab, fontsize=22)
                plt.xlabel('')

                plt.grid(False)

                plt.xlim = xlim
                plt.setp(ax.get_xticklabels(), rotation=0,fontsize=11) 
                plt.setp(ax.get_yticklabels(), fontsize=15)


                plt.savefig(dirOutput + 'plot/' + STATIONS_SHORT[st] + "_" + FREQ[f] + "_plot_" + str(sec) + "sec" + ext +".png", dpi=300, facecolor='w', edgecolor='w', bbox_extra_artists=(lgd,), bbox_inches='tight')

                plt.show()

                plt.close()
        else:
            print('station', STATIONS[st], 'has no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
del sts
del freq
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G3> </a> **Plot daily median with percentiles**            
Output will be put in dirOutput/medianDay folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values to be plotted (0 - 1, default .1 and .9).          
Select whether to plot daily total/average spl or not (Y or N, default N).            
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
prc1 = 0.1 #first percentile, 
prc2 = 0.9 #last percentile

plot_total_spl = 'n' #daily total spl
plot_average_spl = 'n' #daily average spl

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
prc1 = 0.1 #first percentile, 
prc2 = 0.9 #last percentile

plot_total_spl = 'n'
plot_average_spl = 'y'

sts = [1,2]
freq = [63]

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'medianDay') == False:
    os.mkdir(dirOutput + 'medianDay')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
if not 'plot_total_spl' in locals():
    plot_total_spl = 'n'
if not 'plot_average_spl' in locals():
    plot_average_spl = 'n'

if not 'prc1' in locals():
    prc1 = 0.1
if not 'prc2' in locals():
    prc2 = 0.9

a = (t_final - t_init).total_seconds()

for st in sts:
    if st in stations:
        a = (last_time[st] - first_time[st]).total_seconds()
        if a > dayInSec*2: # at least 2 days
            df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
            if len(df) > 0:   

                for f in freq:

                    fig = plt.figure(figsize=(11.7,8.3)) #A4
                    #fig = plt.figure(1) #A4
                    ax = fig.add_subplot(111)

                    timeSpan = '1D'
                    (da,va) = median(df,F[f],timeSpan)
                    ax = plt.plot(da,va,'r-',markersize = 5, markerfacecolor='r', markeredgecolor='r',label='Daily Median')
                    (da,Yq1,Yq2) = perc(df,F[f],timeSpan,prc1,prc2)
                    ax = plt.fill_between(da, Yq1, Yq2, color='r', alpha=.1,label='Daily ' + str(int(prc1*100)) + '-' + str(int(prc2*100)) + 'prc')

                    if plot_total_spl == 'Y' or plot_total_spl == 'y':
                        da,totale,averaged = totalXperiod(df,f,timeSpan)
                        ax = plt.plot(da,totale,'k-o',markersize = 1, markerfacecolor='k', alpha=.5,markeredgecolor='k',label='Daiy Total SPL')

                    if plot_average_spl == 'Y' or plot_average_spl == 'y':
                        timeSpan = '1D'
                        da,totale,averaged = totalXperiod(df,f,timeSpan)
                        ax = plt.plot(da,averaged,'k-o',markersize = 1, markerfacecolor='k', alpha=1,markeredgecolor='k',label='Daily Averaged SPL')

                    SPLT,SPL_av = totalSPL(df,f)
                    if plot_total_spl == 'Y' or plot_total_spl == 'y':
                        la = 'Total SPL = ' + str('{:.2f}')
                        plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10] + ' (' +  la.format(SPLT) + ')', fontsize=15,fontweight = 'bold')
                    elif plot_average_spl == 'Y' or plot_average_spl == 'y':
                        la = 'Averaged SPL = ' + str('{:.2f}')
                        plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10] + ' (' +  la.format(SPL_av) + ')', fontsize=15,fontweight = 'bold')
                    else:
                        plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')


                    ax = plt.gca() #get axes

                    l10 = df[F[f]].quantile(0.9)
                    l90 = df[F[f]].quantile(0.1)
                    l50 = df[F[f]].quantile(0.5)

                    xlim = [mdates.date2num(t_init)-0.3,mdates.date2num(t_final)-0.7]
                    plt.plot([xlim[0],xlim[1]],[l10,l10],'--k',linewidth=.5)
                    plt.plot([xlim[0],xlim[1]],[l90,l90],'--k',linewidth=.5)
                    plt.plot([xlim[0],xlim[1]],[l50,l50],'--k',linewidth=.5,label = '10/50/90 EL')

 

                    if a > 60*dayInSec:
                        ax.xaxis.set_major_locator(MonthLocator())
                        formatter = mdates.DateFormatter('%m/%y') 
                        ax.xaxis.set_major_formatter(formatter)
                    elif a <= 60*dayInSec and a > 15*dayInSec:
                        ax.xaxis.set_major_locator(DayLocator(interval=5))
                        formatter = mdates.DateFormatter('%d/%m') 
                        ax.xaxis.set_major_formatter(formatter)
                    elif a <= 15*dayInSec and a > 1*dayInSec:
                        ax.xaxis.set_major_locator(DayLocator())
                        formatter = mdates.DateFormatter('%d/%m') 
                        ax.xaxis.set_major_formatter(formatter)
                    elif a <= 1*dayInSec and a > .2*dayInSec:
                        ax.xaxis.set_major_locator(HourLocator())
                        formatter = mdates.DateFormatter('%H') 
                        ax.xaxis.set_major_formatter(formatter)
                    else:
                        ax.xaxis.set_major_locator(MinuteLocator(byminute=range(1,61,10)))
                        formatter = mdates.DateFormatter('%H:%M')
                        ax.xaxis.set_major_formatter(formatter)
                       
                    
                    
                    

                    plt.setp(ax.get_xticklabels(), rotation=0,fontsize=11)
                    plt.setp(ax.get_yticklabels(), fontsize=15)

                    if 'ylim' in locals():
                        if len(ylim) > 0:
                            plt.ylim(ylim)
                            
                    

                    plt.ylabel(lab, fontsize=22)


                    handles, labels = ax.get_legend_handles_labels()
                    
                    lgd = ax.legend(handles, labels, loc=1, bbox_to_anchor=(1.27, 1), fontsize=15)
                    
                    plt.grid(False)
                    plt.savefig(dirOutput + 'medianDay/' + STATIONS_SHORT[st] + "_" + FREQ[f] +"_plot_dailyAveraged_" + str(sec) + "sec" + ext +".png", dpi=300, facecolor='w', edgecolor='w', bbox_extra_artists=(lgd,), bbox_inches='tight')
                    plt.show()
                    plt.close()
            else:
                print('station', STATIONS[st], 'has no data')
        else:
            print('too few data')
    else:
        print('station', STATIONS[st], 'not loaded')

del sts
del freq
del prc1
del prc2
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id = G4> </a> **Density plot**               
Output will be put in dirOutput/density folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)     
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select x lim, leave blank for auto.              
**input example:**
```python
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

xlim = [60,90] #x lim, [inf,sup] or [] for auto
```

In [ ]:
sts = []
freq = [63]
xlim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'density') == False:
    os.mkdir(dirOutput + 'density')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
    
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            for f in freq:

                g = sns.kdeplot(df[F[f]]) 

                plt.xlabel(lab,fontsize =15)    
                plt.ylabel('Probability')    
                
                ax = plt.gca() #get axes
                if 'xlim' in locals():
                        if len(xlim) > 0:
                            #plt.xlim(xlim)
                            g.set(xlim=xlim)

                
                plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                plt.grid(False)
                plt.savefig(dirOutput + 'density/' + STATIONS_SHORT[st] + "_" + FREQ[f] +"_density_" + str(sec) + "sec" + ext +".png", dpi=300, facecolor='w', edgecolor='w')
                plt.show()
                plt.close()
           
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
del sts
del freq
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id = G5> </a> **Histogram**    
Output will be put in dirOutput/histo folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)     
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select x lim, leave blank for auto.              
**input example:**
```python
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

xlim = [60,90] #x lim, [inf,sup] or [] for auto
```

In [ ]:
sts = []
freq = [63]

xlim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'histo') == False:
    os.mkdir(dirOutput + 'histo')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
       
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            for f in freq:
                if f in F:
                
                    g = df[F[f]].plot(kind = 'hist',bins = 20)
                    plt.xlabel(lab, fontsize=15)    
                    plt.ylabel('Count')    
                    if 'xlim' in locals():
                        if len(xlim) > 0:
                            #plt.xlim(xlim)
                            g.set(xlim=xlim)
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                    plt.grid(False)
                    plt.savefig(dirOutput + 'histo/' + STATIONS_SHORT[st] + "_" + FREQ[f] +"_histo_" + str(sec) + "sec" + ext +".png", dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                    print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
del sts
del freq
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G6> </a> **Boxplot of the distribution**       
Output will be put in dirOutput/boxPlot folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = []
freq = [63]

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'boxPlot') == False:
    os.mkdir(dirOutput + 'boxPlot')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
       

if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90
    
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            for f in freq:
                if f in F:
                    medianprops = dict(linestyle='--', linewidth=2.5, color='red',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')
                    medianpointprops = dict(marker='+', markersize = 3,markeredgecolor='red',
                                  markerfacecolor='red')


                    meanpointprops = dict(marker='o', markersize = 3,markeredgecolor='black',
                                  markerfacecolor='black')


                    sns.set(style="whitegrid")
                    ax = sns.boxplot(y = F[f],color='w',linewidth=1.5,whis = (whi1,whi2),showmeans = False,meanprops = meanpointprops,
                        medianprops = medianprops, showfliers = False, data=df)
                    plt.xticks(fontsize=8)
                    plt.ylabel(lab,fontsize=15)
                    if 'ylim' in locals():
                            if len(ylim) > 0:
                                plt.ylim(ylim)
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                    plt.grid(True)
                    plt.legend()
                    plt.savefig(dirOutput + 'boxPlot/' + STATIONS_SHORT[st] + "_" + FREQ[f] +'_boxplot' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                    print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
del sts
del whi1
del whi2
del freq
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G7> </a> **Violinplot of the distribution**    
Output will be put in dirOutput/violinPlot folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = []
freq = [125]

ylim = []


In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'violinPlot') == False:
    os.mkdir(dirOutput + 'violinPlot')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90
    
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            for f in freq:
                if f in F:   
                    # non-parametric pdf
                    valXkde = df[F[f]].values
                    cleanedList = [v for v in valXkde if np.isnan(v) == False]
                    nparam_density = stats.kde.gaussian_kde(cleanedList)
                    x = np.linspace(60, 150, 200)
                    nparam_density = nparam_density(x)
                    pm = max(nparam_density)

                    sns.set(style="whitegrid")
                    medianprops = dict(linestyle='', linewidth=1.5, color='k',label = '_nolegend_')

                    ax = sns.boxplot(y = F[f],color='k',linewidth=1.5,whis = (whi1,whi2),showmeans = False,medianprops = medianprops,
                        width=0.3, showfliers = False,showbox = False, data=df) # medianprops = medianprops,
                    ax = sns.violinplot(y = F[f],linewidth=1,color='w',inner = 'box',data=df)#inner = 'quartile'

                    la = str('{:.3f}') 
                    
                    if 'ylim' in locals():
                            if len(ylim) > 0:
                                plt.ylim(ylim)
                    yl = ax.get_ylim()
                    plt.setp(ax.get_xticklabels(),fontsize=15)
                    plt.setp(ax.get_yticklabels(), fontsize=15)
                    plt.ylabel(lab, fontsize=15)
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                    plt.grid(True)
                    plt.savefig(dirOutput + 'violinPlot/' + STATIONS_SHORT[st] + "_" + FREQ[f] +'_Violinplot' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                    print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
del sts
del freq
del whi1
del whi2
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G8> </a> **Plot of percentiles values for a frequecy in the selected period**      
Output will be put in dirOutput/percPlot folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select x lim, leave blank for auto.              
**input example:**
```python
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

xlim = [60,90] #x lim, [inf,sup] or [] for auto
```

In [ ]:
sts = []
freq = [125]

xlim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'percPlot') == False:
    os.mkdir(dirOutput + 'percPlot')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            for f in freq:
                if f in F:
                    percen = []
                    for i in range(0,101):
                        percen.append(np.nanpercentile(df[F[f]],i))

                    y = list(range(100,-1,-1))    

                    plt.plot((percen),y)

                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                    plt.xlabel(lab, fontsize=10)
                    plt.ylabel('Exceedance Level', fontsize=15)
                    ax = plt.gca() #get axes


                    plt.ylim([0,100])
                    if 'xlim' in locals():
                            if len(xlim) > 0:
                                #plt.ylim(ylim)
                                ax.set_xlim(xlim)
                    xlim = ax.get_xlim() #use these or next plt.xlim

                    plt.plot([xlim[0],xlim[1]],[10,10],'--k',linewidth=.5)
                    plt.plot([xlim[0],xlim[1]],[90,90],'--k',linewidth=.5)
                    plt.plot([xlim[0],xlim[1]],[50,50],'--k',linewidth=.5,label = '10/50/90 Level')

                    plt.setp(ax.get_xticklabels(),fontsize=15)
                    plt.setp(ax.get_yticklabels(), fontsize=15)

                    plt.legend()
                    plt.grid(False)
                    plt.savefig(dirOutput + 'percPlot/' + STATIONS_SHORT[st] + "_" + FREQ[f] +'_percentile' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                    print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
del sts
del freq
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G9> </a> **Boxplot of day and night data**                
Output will be put in dirOutput/boxDN folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select hours of start and end of days (default 7 and 20, that means day hours are from 7 to 19:59:59 and night hours from 20 to 6:59:59)                 
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

day_i = 7 #start of day hour
day_f = 20 #end of day hour

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

day_i = 6 #7
day_f = 16 #18

sts = []
freq = [250]

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'boxDN') == False:
    os.mkdir(dirOutput + 'boxDN')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
    
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90
    
if not 'day_i' in locals():
    day_i = 7
if not 'day_f' in locals():
    day_f = 20
    
for st in sts:
    if st in stations:
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            warnings.filterwarnings("ignore")
            df.loc[(df.index.hour >= day_i) & (df.index.hour <day_f), "Time"] = "Day"
            df.loc[(df.index.hour < day_i) | (df.index.hour >=day_f), "Time"] = "Night"
            dfU_D = df[df['Time'] == "Day"]
            dfU_N = df[df['Time'] == "Night"]
            
            for f in freq:
                if f in F:
                    
                    # U test
                    dD = dfU_D[F[f]].dropna()
                    dN = dfU_N[F[f]].dropna()
                    print("st: " + str(st) + " - f: " + F[f])
                    print(stats.mannwhitneyu(dD,dN))
                    print(stats.kstest(dD,dN))
                    print(stats.kruskal(dD,dN))
                    
                    medianprops = dict(linestyle='--', linewidth=2.5, color='red',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')
                    medianpointprops = dict(marker='+', markersize = 3,markeredgecolor='red',
                                  markerfacecolor='red')


                    meanpointprops = dict(marker='o', markersize = 3,markeredgecolor='black',
                                  markerfacecolor='black')


                    sns.set(style="whitegrid")
                    ax = sns.boxplot(x = 'Time',y = F[f],color='w',linewidth=1.5,whis = (whi1,whi2),showmeans = False,meanprops = meanpointprops,
                        medianprops = medianprops, showfliers = False, order=["Day", "Night"],data=df)

                    plt.xticks(fontsize=8)

                    plt.ylabel(lab, fontsize=15)

                    if 'ylim' in locals():
                            if len(ylim) > 0:
                                plt.ylim(ylim)

                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                    
                    plt.grid(True)

                    plt.savefig(dirOutput + 'boxDN/' + STATIONS_SHORT[st] + "_" + FREQ[f] +'_boxDN' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                    print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
    
if 'day_i' in locals():
    del day_i
if 'day_f' in locals():
    del day_f
del st
del freq
    

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G10> </a> **Violinplot of day and night data**             
Output will be put in dirOutput/vioDN folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select hours of start and end of days (default 7 and 20, that means day hours are from 7 to 19:59:59 and night hours from 20 to 6:59:59)                 
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

day_i = 7 #start of day hour
day_f = 20 #end of day hour

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

day_i = 6 #7
day_f = 16 #18

sts = []
freq = [250]

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'vioDN') == False:
    os.mkdir(dirOutput + 'vioDN')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
    
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90
    
if not 'day_i' in locals():
    day_i = 7
if not 'day_f' in locals():
    day_f = 20
    
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            warnings.filterwarnings("ignore")
            df.loc[(df.index.hour >= day_i) & (df.index.hour <day_f), "Time"] = "Day"
            df.loc[(df.index.hour < day_i) | (df.index.hour >=day_f), "Time"] = "Night"
            dfU_D = df[df['Time'] == "Day"]
            dfU_N = df[df['Time'] == "Night"]

            for f in freq:
                if f in F:
                    # U test
                    # U test
                    dD = dfU_D[F[f]].dropna()
                    dN = dfU_N[F[f]].dropna()
                    print("st: " + str(st) + " - f: " + F[f])
                    print(stats.mannwhitneyu(dD,dN))
                    print(stats.kstest(dD,dN))
                    print(stats.kruskal(dD,dN))
                    
                    sns.set(style="whitegrid")
                    
                    medianprops = dict(linestyle='', linewidth=1.5, color='k',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')

                    sns.boxplot(x = 'Time',y = F[f],linewidth=1.5,whis = (whi1,whi2),showmeans = False,medianprops = medianprops,
                        width=0.3, showfliers = False,showbox = False, order=["Day", "Night"],data=df)

                    sns.violinplot(x = 'Time', y = F[f],color='w', linewidth=1, inner = 'box',order=["Day", "Night"],data=df)
                    if 'ylim' in locals():
                            if len(ylim) > 0:
                                plt.ylim(ylim)
                    plt.xticks(fontsize=8)
                    plt.xticks(fontsize=14)
                    plt.ylabel(lab,fontsize=15)
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
                    plt.grid(True)
                    plt.savefig(dirOutput + 'vioDN/' + STATIONS_SHORT[st] + "_" + FREQ[f] +'_vioDN' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                    print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
    
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
    
if 'day_i' in locals():
    del day_i
if 'day_f' in locals():
    del day_f

del st
del freq

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G11> </a> **Plot of the median values of each frequencies in the selected period with percentiles band**     
Output will be put in dirOutput/medianFreq folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values to plot (0 - 1, default .1 and .9).          
Select stations (leave it empty to use all loaded stations)                            
Select y lim, leave blank for auto.              
**input example:**
```python
prc1 = .1 #first percentile (first whiskers value) 
prc2 = .9 #last percentile (second whiskers value)

sts = [1,2,5]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
prc1 = 0.1 #first percentile (first whiskers value) 
prc2 = 0.9 #last percentile (second whiskers value)

sts = []

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'medianFreq') == False:
    os.mkdir(dirOutput + 'medianFreq')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
    
if not 'prc1' in locals():
    prc1 = 0.1
if not 'prc2' in locals():
    prc2 = 0.9
    
for st in sts:
    if st in stations:
        
        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            #List of frequencies
            ff = [10,12.5,16,20,25,31.5,40,50,63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,
                10000,12500,16000,20000]

            mediana = []
            percenMin = []
            percenMax = []
            percen25 = []
            percen75 = []
            percen1 = []
            percen99 = []
            splTot = []
            splav = []
            mean = []

            fig = plt.figure(figsize=(11.7,8.3)) #A4
            print(STATIONS[st])
            for f in ff:
                
                mediana.append((df[F[f]].median()))#(np.nanpercentile(spl,0.5))#
                percenMin.append(df[F[f]].quantile(prc1))
                percenMax.append(df[F[f]].quantile(prc2))
                percen75.append(df[F[f]].quantile(0.75))
                percen25.append(df[F[f]].quantile(0.25))
                percen1.append(df[F[f]].quantile(0.99))
                percen99.append(df[F[f]].quantile(0.01))
                SPLT,SPL_av = totalSPL(df,f)
                mean.append(SPL_av)
                
                la = FREQ[f] + ': ' + str('{:.2f}')
                #print(la.format(df[F[f]].median()))

                #SPLT,SPL_av = totalSPL(df,f)
                #splTot.append(SPLT)
                #splav.append(SPL_av) 

            medianaToKeep = mediana.copy()

            xx = [mat.log10(10),mat.log10(12.5),mat.log10(16),mat.log10(20),mat.log10(25),mat.log10(31.5),mat.log10(40),mat.log10(50),mat.log10(63),mat.log10(80),
                    mat.log10(100),mat.log10(125),mat.log10(160),mat.log10(200),mat.log10(250),mat.log10(315),mat.log10(400),mat.log10(500),mat.log10(630),mat.log10(800),
                    mat.log10(1000),mat.log10(1250),mat.log10(1600),mat.log10(2000),mat.log10(2500),mat.log10(3150),mat.log10(4000),mat.log10(5000),mat.log10(6300),mat.log10(8000),
                    mat.log10(10000),mat.log10(12500),mat.log10(16000),mat.log10(20000)]

            #ax = plt.plot(xx,splTot,'ko-',markersize=3,label='Total SPL')
            #ax = plt.plot(xx,splav,'ko-',markersize=3,label='Averaged SPL')

            ax = plt.plot(xx,mediana,'bo-',markersize=3,label='median')
            #ax = plt.plot(xx,mean,'r--',markersize=3,label='mean')
            ax = plt.fill_between(xx, percenMin, percenMax, color='r', alpha=.1,label=str(int(prc1*100)) + '-' + str(int(prc2*100)) + 'EL')
            ax = plt.fill_between(xx, percen25, percen75, color='g', alpha=.1,label=str(int(0.25*100)) + '-' + str(int(0.75*100)) + 'EL')
            ax = plt.plot(xx,percen1,'k--',markersize=3,label=' 1 EL')
            ax = plt.plot(xx,percen99,'k--',markersize=3,label='99 EL')

            ax = plt.gca() #get axes

            handles, labels = ax.get_legend_handles_labels()
            lgd = ax.legend(handles, labels, loc=1, bbox_to_anchor=(1.27, 1), fontsize=15)


            xx1 = [mat.log10(10),mat.log10(16),mat.log10(25),mat.log10(40),mat.log10(50),mat.log10(63),mat.log10(80),mat.log10(125),mat.log10(200),mat.log10(315),mat.log10(500),mat.log10(800),
                    mat.log10(1250),mat.log10(2000),mat.log10(3150),mat.log10(5000),mat.log10(8000),
                    mat.log10(12500),mat.log10(20000)]
            xti = ('10','16','25','40','50','63','80','125','200','315','500','800','1250','2000','3150','5000','8000','12500','20000')

            #plt.ylim(65,110)

            plt.xticks(xx1, xti)
            plt.xticks(fontsize=14)
            plt.yticks(fontsize=14)
            plt.ylabel(lab,fontsize=15)
            plt.xlabel('1/3 OCTAVE FILTER CENTER FREQUENCIES [Hz]',fontsize=15)
            if 'ylim' in locals():
                if len(ylim) > 0:
                    plt.ylim(ylim)
            plt.title(STATIONS_SHORT[st] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
            plt.legend()
            plt.grid(True,linestyle='dotted', linewidth=.5)
            plt.savefig(dirOutput + 'medianFreq/' + STATIONS_SHORT[st] + '_Frequecnies_AVERAGED' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            plt.show()
            plt.close()
            del ff
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
        
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'prc1' in locals():
    del prc1
if 'prc2' in locals():
    del prc2
del st
del freq

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G12> </a> **Comparing percentile values of different frequecies in the selected period**     
Output will be put in dirOutput/perConfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values to plot (0 - 1, default .1 and .9).          
Select stations (leave it empty to use all loaded stations)                            
Select y lim, leave blank for auto.              
**input example:**
```python
sts = [1,2,5]
freq = [63, 125, 250, 4000] #frequencies to be compared, if [] default [63, 125, 250, 4000]

xlim = [60,90] #x lim, [inf,sup] or [] for auto
```

In [ ]:
sts = []
freq = [125,250]

xlim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'perConfr') == False:
    os.mkdir(dirOutput + 'perConfr')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
y = list(range(100,-1,-1))    

for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:

            for f in freq:
                if f in F:
                    percen = []
                    for i in range(0,101):
                        percen.append(df[F[f]].quantile(i/100))

                    plt.plot((percen),y,label=FREQ[f])
                else:
                    print('Frequency ', f, ' not in Frequencies list')

            plt.title(STATIONS_SHORT[st] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
            plt.xlabel(lab, fontsize=10)
            plt.ylabel('Exceedance Level', fontsize=15)       

            ax = plt.gca() #get axes
            if 'xlim' in locals():
                if len(xlim) > 0:
                    ax.set_xlim(xlim)
                    
            xlim1 = ax.get_xlim()
            
            plt.plot([xlim1[0],xlim1[1]],[10,10],'--k',linewidth=.5)
            plt.plot([xlim1[0],xlim1[1]],[90,90],'--k',linewidth=.5)
            plt.plot([xlim1[0],xlim1[1]],[50,50],'--k',linewidth=.5,label = '10/50/90 prc') 
            ax.set_xlim(xlim1)
            plt.legend()
            plt.grid(False)
            plt.savefig(dirOutput + 'perConfr/' + STATIONS_SHORT[st] + '_percentiles_compare' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            plt.show()
            plt.close()

        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')

if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'prc1' in locals():
    del prc1
if 'prc2' in locals():
    del prc2
del st
del freq

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G13> </a> **Boxplot of different frequecies in the selected period**          
Output will be put in dirOutput/boxPlotCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = []
freq = [63,125]

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'boxPlotCfr') == False:
    os.mkdir(dirOutput + 'boxPlotCfr')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
             
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90
for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        
        if len(df) > 0:
            vari = []
            variF = []
            for f in freq:
                if f not in F:
                    print('Frequency ', f, ' not in Frequencies list')
                    continue

                vari.append(F[f])
                variF.append(FREQ[f])

            medianprops = dict(linestyle='--', linewidth=2.5, color='red')
            medianpointprops = dict(marker='+', markersize = 3,markeredgecolor='red',
                          markerfacecolor='red')


            meanpointprops = dict(marker='o', markersize = 3,markeredgecolor='black',
                          markerfacecolor='black')

            sns.set(style="whitegrid")
            ax = sns.boxplot(x="variable", y="value",color='w',linewidth=1.5,whis = (whi1,whi2),showmeans = False,meanprops = meanpointprops,
                medianprops = medianprops, showfliers = False, data=pd.melt(df,value_vars=vari))
            ax.set_xticklabels(variF)
            plt.xticks(fontsize=8)
            if 'ylim' in locals():
                if len(ylim) > 0:
                    plt.ylim(ylim)
            plt.ylabel(lab,fontsize=15)
            plt.xlabel('')
            plt.title(STATIONS_SHORT[st] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
            plt.grid(True)
            plt.savefig(dirOutput + 'boxPlotCfr/' + STATIONS_SHORT[st] + '_box_cfr_freq' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            plt.show()
            plt.close()
            
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')

if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
del st
del freq

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G14> </a> **Violinplot of different frequecies in the selected period**                
Output will be put in dirOutput/violinPlotCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)                
Select y lim, leave blank for auto.              
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = []
freq = [63,125]

ylim = [50,150]

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'violinPlotCfr') == False:
    os.mkdir(dirOutput + 'violinPlotCfr')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
                 
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90

for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            vari = []
            variF = []

            for f in freq:
                if f not in F:
                    print('Frequency ', f, ' not in Frequencies list')
                    continue
                vari.append(F[f])
                variF.append(FREQ[f])

            sns.set(style="whitegrid")

            medianprops = dict(linestyle='', linewidth=1.5, color='k',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')

            sns.boxplot(x = "variable", y="value",linewidth=1.5,whis = (whi1,whi2),showmeans = False,medianprops = medianprops, 
                          width=0.3, showfliers = False,showbox = False,data=pd.melt(df,value_vars=vari))

            ax = sns.violinplot(x="variable", y="value",color='w',linewidth=1,inner = 'box', data=pd.melt(df,value_vars=vari))
            ax.set_xticklabels(variF)
            plt.xticks(fontsize=8)
            plt.xlabel('')
            plt.title(STATIONS_SHORT[st] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
            if 'ylim' in locals():
                if len(ylim) > 0:
                    plt.ylim(ylim)
            plt.ylabel(lab, fontsize=10)
            plt.grid(True)

            plt.savefig(dirOutput + 'violinPlotCfr/' + STATIONS_SHORT[st] + '_Violinplot_cfr_freq' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            plt.show()
            plt.close()
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')
    
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
del st
del freq

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G15> </a> **Boxplot of monthly data**          
Output will be put in dirOutput/BoxMonthlyCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select the months of the years to plot 
Select y lim, leave blank for auto.                 
Select the months and the years to take in account. They must be between t_init and t_final.      
i.g. to plot June, July and August for both 2020 and 2021 set
```python
month = [6,7,8]
year = [2020,2021]
```
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

mesi = {2020:[3,4,5,6,7,8,9,10,11,12],2021:[1,2,3]} #months to be plotted

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1]
freq = [4000]

mesi = {2020:[3,6],2021:[3,6]}

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'BoxMonthlyCfr') == False:
    os.mkdir(dirOutput + 'BoxMonthlyCfr')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
                     
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90

if 'mesi' in locals():
    
    for st in sts:
        if st in stations:

            df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
            if len(df) > 0:

                for f in freq:
                    if f in F:
                        warnings.filterwarnings("ignore")
                        for y in mesi:
                            for m in mesi[y]:
                                
                                if m < 10:
                                    df.loc[(df.index.month == m) & (df.index.year == y), "Mo"] = '0' + str(m) + '/' + str(y)[-2:]
                                else:
                                    df.loc[(df.index.month == m) & (df.index.year == y), "Mo"] = str(m) + '/' + str(y)[-2:]
                                    
                                    
                                dtest = df.loc[(df.index.month == m) & (df.index.year == y), "Mo"]        
                                if len(dtest) == 0:
                                    if m < 10:
                                        df1 = pd.DataFrame([[np.nan,'0' + str(m) + '/' + str(y)[-2:]]], columns=list([F[f],'Mo']), index=[datetime(y, m, 15, hour=0, minute=0, second=0)])
                                        df= pd.concat([df, df1]) 
                                    else:
                                        df1 = pd.DataFrame([[np.nan,str(m) + '/' + str(y)[-2:]]], columns=list([F[f],'Mo']), index=[datetime(y, m, 15, hour=0, minute=0, second=0)])
                                        df= pd.concat([df, df1]) 
                        
                        df = df.sort_index()
                                
                                

                        medianprops = dict(linestyle='--', linewidth=2.5, color='red')
                        medianpointprops = dict(marker='+', markersize = 3,markeredgecolor='red',
                                      markerfacecolor='red')


                        meanpointprops = dict(marker='o', markersize = 3,markeredgecolor='black',
                                      markerfacecolor='black')

                        sns.set(style="whitegrid")
                        ax = sns.boxplot(x="Mo", y=F[f],color='w',linewidth=1.5,whis = (whi1,whi2), showmeans = False,meanprops = meanpointprops,
                            medianprops = medianprops, showfliers = False, data=df)


                        plt.xticks(fontsize=8)
                        if 'ylim' in locals():
                            if len(ylim) > 0:
                                plt.ylim(ylim)
                        plt.ylabel(lab,fontsize = 15)
                        plt.xlabel('')
                        plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f], fontsize=15,fontweight = 'bold')

                        plt.grid(True)
                        plt.savefig(dirOutput + 'BoxmonthlyCFR/' + STATIONS_SHORT[st] + "_" + FREQ[f] + '_Boxplot_monthly' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                        plt.show()
                        plt.close()
                    else:
                        print('Frequency ', f, ' not in Frequencies list')

            else:
                print('station', STATIONS[st], 'no data')
        else:
            print('station', STATIONS[st], 'not loaded')

    if 'ylim' in locals():
        del ylim
    if 'xlim' in locals():
        del xlim

    if 'whi1' in locals():
        del whi1
    if 'whi2' in locals():
        del whi2
    
    del dtest
    if 'df1' in locals():
        del df1
    del mesi
    del st
    del freq
else:
    print('no month(s)/year(s) defined')

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G16> </a> **Violinplot of monthly data**    
Output will be put in dirOutput/ViolinMonthlyCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select the months of the years to plot 
Select y lim, leave blank for auto.                 
Select the months and the years to take in account. They must be between t_init and t_final.      
i.g. to plot June, July and August for both 2020 and 2021 set
```python
mesi = {2020:[6,7,8],2021:[6,7,8]}
```
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

mesi = {2020:[3,4,5,6,7,8,9,10,11,12],2021:[1,2,3]} # select  year(s) and month(s)

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [6]
freq = []
mesi = {2020:[3,4,5,6,7,8,9,10,11,12],2021:[1,2,3,4,5,6]}

ylim = [50,150]

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'ViolinMonthlyCfr') == False:
    os.mkdir(dirOutput + 'ViolinMonthlyCfr')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
         
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90

if 'mesi' in locals():
    
    for st in sts:
        if st in stations:

            df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
            if len(df) > 0:
                
                for f in freq:
                    if f in F:
                        warnings.filterwarnings("ignore")
                        for y in mesi:
                            for m in mesi[y]:
                                
                                if m < 10:
                                    df.loc[(df.index.month == m) & (df.index.year == y), "Mo"] = '0' + str(m) + '/' + str(y)[-2:]
                                else:
                                    df.loc[(df.index.month == m) & (df.index.year == y), "Mo"] = str(m) + '/' + str(y)[-2:]
                                    
                                    
                                dtest = df.loc[(df.index.month == m) & (df.index.year == y), "Mo"]        
                                if len(dtest) == 0:
                                    if m < 10:
                                        df1 = pd.DataFrame([[np.nan,'0' + str(m) + '/' + str(y)[-2:]]], columns=list([F[f],'Mo']), index=[datetime(y, m, 15, hour=0, minute=0, second=0)])
                                        df= pd.concat([df, df1]) 
                                    else:
                                        df1 = pd.DataFrame([[np.nan,str(m) + '/' + str(y)[-2:]]], columns=list([F[f],'Mo']), index=[datetime(y, m, 15, hour=0, minute=0, second=0)])
                                        df= pd.concat([df, df1]) 
                        
                        df = df.sort_index()
                        
                        sns.set(style="whitegrid")

                        medianprops = dict(linestyle='', linewidth=1.5, color='k',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')

                        sns.boxplot(x = 'Mo',y = F[f],linewidth=1.5,whis = (whi1,whi2),showmeans = False,medianprops = medianprops,
                            width=0.3, showfliers = False,showbox = False, data=df)

                        ax = sns.violinplot(x = 'Mo', y = F[f],color='w', linewidth=1, inner = 'box',data=df)

                        plt.ylabel(lab, fontsize=10)
                        plt.xlabel('Month')
                        plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f], fontsize=15,fontweight = 'bold')
                        if 'ylim' in locals():
                            if len(ylim) > 0:
                                plt.ylim(ylim)
                        plt.xticks(fontsize=8)
                        plt.setp(ax.get_xticklabels(),fontsize=8)
                        plt.setp(ax.get_yticklabels(), fontsize=15)
                        plt.grid(True)
                        plt.savefig(dirOutput + 'ViolinMonthlyCfr/' + STATIONS_SHORT[st] + "_" + FREQ[f] + '_Violinplot_monthly' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                        plt.show()
                        plt.close()
                    else:
                        print('Frequency ', f, ' not in Frequencies list')
            else:
                print('station', STATIONS[st], 'no data')
        else:
            print('station', STATIONS[st], 'not loaded')

    if 'ylim' in locals():
        del ylim
    if 'xlim' in locals():
        del xlim

    if 'whi1' in locals():
        del whi1
    if 'whi2' in locals():
        del whi2
    
    del dtest
    if 'df1' in locals():
        del df1
    del mesi
    del st
    del freq
    
else:
    print('no month(s)/year(s) defined')

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G17> </a> **Boxplot of different periods**           
Output will be put in dirOutput/BoxPeriodCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select periods to plot 
Select y lim, leave blank for auto.                 
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

Periods = [['2020-06-20','2020-06-21','1'],['2020-06-22','2020-06-23','2'], #Periodos must be defined: ['Start P1','End P1','Label P1'] 
           ['2020-06-24','2020-06-25','3'],['2020-06-26','2020-06-27','4']]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1,2]
freq = [63]

Periods = [['2021-06-1','2021-06-2','Jun-1'],['2021-06-3','2021-06-4','Jun-3'],
           ['2021-06-5','2021-06-6','Jun-5'],['2021-06-7','2021-06-8','Jun-7']]
ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'BoxPeriodCfr') == False:
    os.mkdir(dirOutput + 'BoxPeriodCfr')
    
if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90

if not 'Periods' in locals():
    Periods = [['2020-03-21','2020-06-20','Spring'],['2020-06-21','2020-09-20','Summer'],
           ['2020-09-21','2020-12-20','Autumn'],['2020-12-21','2021-02-28','Winter']]
    
for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:

            for f in freq:
                if f in F:
                    warnings.filterwarnings("ignore")

                    df_P = pd.DataFrame([])
                    for p in Periods:
                        cc = (df.index >=  p[0]) & (df.index <  p[1])
                        df.loc[cc, "P"] = p[2]
                        print(p[2], '50th percentile: ', df.loc[cc][F[f]].quantile(0.5))
                        df_P= pd.concat([df_P, df.loc[cc]])#df_P.append(df.loc[cc])

                    medianprops = dict(linestyle='--', linewidth=2.5, color='red',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')
                    medianpointprops = dict(marker='+', markersize = 3,markeredgecolor='red',
                                  markerfacecolor='red')


                    meanpointprops = dict(marker='o', markersize = 3,markeredgecolor='black',
                                  markerfacecolor='black')


                    sns.set(style="whitegrid")
                    ax = sns.boxplot(x = 'P',y = F[f],color='w',linewidth=1.5,whis = (whi1,whi2),showmeans = False,meanprops = meanpointprops,
                        medianprops = medianprops, showfliers = False, data=df_P)

                    plt.xticks(fontsize=8)
                    if 'ylim' in locals():
                        if len(ylim) > 0:
                            plt.ylim(ylim)
                    plt.ylabel(lab,fontsize = 15)
                    plt.xlabel('')
                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f], fontsize=15,fontweight = 'bold')
                    plt.grid(True)
                    plt.savefig(dirOutput + 'BoxPeriodCFR/' + STATIONS_SHORT[st] + "_" + FREQ[f] + '_Boxlot_period' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                        print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')

if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
if 'df_P' in locals():
    del df_P
        

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G18> </a> **Violinplot of different periods**       
Output will be put in dirOutput/ViolinPeriodCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select periods to plot 
Select y lim, leave blank for auto.                 
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

Periods = [['2020-06-20','2020-06-21','1'],['2020-06-22','2020-06-23','2'], #Periodos must be defined: ['Start P1','End P1','Label P1'] 
           ['2020-06-24','2020-06-25','3'],['2020-06-26','2020-06-27','4']]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1,2]
freq = [63]

Periods = [['2021-06-1','2021-06-2','Jun-1'],['2021-06-3','2021-06-4','Jun-3'],
           ['2021-06-5','2021-06-6','Jun-5'],['2021-06-7','2021-06-8','Jun-7']]

ylim = [50,120]

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'ViolinPeriodCfr') == False:
    os.mkdir(dirOutput + 'ViolinPeriodCfr')
    
if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
    
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90

if not 'Periods' in locals():
    Periods = [['2020-03-21','2020-06-20','Spring'],['2020-06-21','2020-09-20','Summer'],
           ['2020-09-21','2020-12-20','Autumn'],['2020-12-21','2021-02-28','Winter']]
    
for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:

            for f in freq:
                if f in F:
                    warnings.filterwarnings("ignore")
                    df_P = pd.DataFrame([])
                   
                    for p in Periods:
                        cc = (df.index >=  p[0]) & (df.index < p[1])
                        df.loc[cc, "P"] = p[2]
                        print(p[2], '50th percentile: ', df.loc[cc][F[f]].quantile(0.5))
                        
                        df_P= pd.concat([df_P, df.loc[cc]])#df_P.append(df.loc[cc])
                        
                   
                    sns.set(style="whitegrid")

                    medianprops = dict(linestyle='', linewidth=1.5, color='k',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')

                    sns.boxplot(x = 'P',y = F[f],linewidth=1.5,whis = (whi1,whi2),showmeans = False,medianprops = medianprops,
                        width=0.3, showfliers = False,showbox = False, data=df_P)

                    ax = sns.violinplot(x = 'P', y = F[f],color='w', linewidth=1, inner = 'box',data=df_P)

                    plt.title(STATIONS_SHORT[st] + ' - ' + FREQ[f], fontsize=15,fontweight = 'bold')
                    if 'ylim' in locals():
                        if len(ylim) > 0:
                            plt.ylim(ylim)
                    plt.xticks(fontsize=8)
                    plt.setp(ax.get_xticklabels(),fontsize=8)
                    plt.setp(ax.get_yticklabels(), fontsize=15)
                    plt.ylabel(lab) 
                    plt.xlabel('')
                    plt.grid(True)
                    plt.savefig(dirOutput + 'ViolinPeriodCfr/' + STATIONS_SHORT[st] + "_" + FREQ[f] + '_Violinplot_period' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    plt.show()
                    plt.close()
                else:
                        print('Frequency ', f, ' not in Frequencies list')
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')

if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
if 'df' in locals():
    del df
if 'df_P' in locals():
    del df_P
    

<a id =G180> </a> **Percentiles of different periods**       
Output will be put in dirOutput/perConfrPeiod folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)                            
Select frequency
Select periods to plot 
Select x lim, leave blank for auto.                 
**input example:**
```python
sts = [1,2,5]
freq = [4000] #Frequency to plot

Periods = [['2020-06-20','2020-06-21','1'],['2020-06-22','2020-06-23','2'],  
           ['2020-06-24','2020-06-25','3'],['2020-06-26','2020-06-27','4']]
#Periodos must be defined: ['Start P1','End P1','Label P1']

xlim = [60,90] #x lim, [inf,sup] or [] for auto
```

In [ ]:
sts = [1,2]
freq = [63]

Periods = [['2021-06-1','2021-06-2','Jun-1'],['2021-06-3','2021-06-4','Jun-3'],
           ['2021-06-5','2021-06-6','Jun-5'],['2021-06-7','2021-06-8','Jun-7']]

xlim = [50,150]

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'perConfrPeiod') == False:
    os.mkdir(dirOutput + 'perConfrPeiod')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations

if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63]
        print('Warning... bad frequency selected, put 63 Hz')
    if len(freq) > 1:
        freq = [63]
        print('Warning... bad frequency selected, put 63 Hz')
else:
    freq = [63]
    print('Warning... bad frequency selected, put 63 Hz')

if not 'Periods' in locals():
    Periods = [['2020-03-21','2020-06-20','Spring'],['2020-06-21','2020-09-20','Summer'],
           ['2020-09-21','2020-12-20','Autumn'],['2020-12-21','2021-02-28','Winter']]
    
y = list(range(100,-1,-1))    

for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:
            df_P = pd.DataFrame([])
            for p in Periods:
                cc = (df.index >=  p[0]) & (df.index < p[1])
                
                df_P= pd.concat([df_P, df.loc[cc]])#df_P.append(df.loc[cc])

                for f in freq:
                    if f in F:
                        print(p[2], '50th percentile: ', df.loc[cc][F[f]].quantile(0.5))
                        percen = []
                        for i in range(0,101):
                            percen.append(df_P[F[f]].quantile(i/100))

                        plt.plot((percen),y,label=p[2])
                    else:
                        print('Frequency ', f, ' not in Frequencies list')

            plt.title(STATIONS_SHORT[st]  + ' ' + FREQ[f] + ' | ' + str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
            plt.xlabel(lab, fontsize=10)
            plt.ylabel('Exceedance Level', fontsize=15)       

            ax = plt.gca() #get axes
            if 'xlim' in locals():
                if len(xlim) > 0:
                    ax.set_xlim(xlim)

            xlim1 = ax.get_xlim()

            plt.plot([xlim1[0],xlim1[1]],[10,10],'--k',linewidth=.5)
            plt.plot([xlim1[0],xlim1[1]],[90,90],'--k',linewidth=.5)
            plt.plot([xlim1[0],xlim1[1]],[50,50],'--k',linewidth=.5,label = '10/50/90 prc') #label = '10/50/75 prc'
            ax.set_xlim(xlim1)
            plt.legend()
            plt.grid(False)
            plt.savefig(dirOutput + 'perConfrPeiod/' + STATIONS_SHORT[st] + '_' + FREQ[f] + '_percentiles_compare' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            plt.show()
            plt.close()

        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')

if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'prc1' in locals():
    del prc1
if 'prc2' in locals():
    del prc2
del st
del freq
if 'df_P' in locals():
    del df_P   

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G19> </a> **Compare the median values of each frequencies in the selected periods**          
Output will be put in dirOutput/MedianPeriodCfr folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)                            
Select periods to compare.
Select y lim, leave blank for auto.                 
**input example:**
```python
sts = [1,2,5]

Periods = [['2020-06-20','2020-06-21','1'],['2020-06-22','2020-06-23','2'], #Periodos must be defined: ['Start P1','End P1','Label P1'] 
           ['2020-06-24','2020-06-25','3'],['2020-06-26','2020-06-27','4']]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
sts = [1,2]

Periods = [['2021-06-1','2021-06-2','Jun-1'],['2021-06-3','2021-06-4','Jun-3'],
           ['2021-06-5','2021-06-6','Jun-5'],['2021-06-7','2021-06-8','Jun-7']]

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'MedianPeriodCfr') == False:
    os.mkdir(dirOutput + 'MedianPeriodCfr')
    
if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
            
if not 'Periods' in locals():
    Periods = [['2020-03-21','2020-06-20','Spring'],['2020-06-21','2020-09-20','Summer'],
           ['2020-09-21','2020-12-20','Autumn'],['2020-12-21','2021-02-28','Winter']]

#List of frequencies
ff = [63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,
    10000,12500,16000,20000]

xx = [mat.log10(63),mat.log10(80),
        mat.log10(100),mat.log10(125),mat.log10(160),mat.log10(200),mat.log10(250),mat.log10(315),mat.log10(400),mat.log10(500),mat.log10(630),mat.log10(800),
        mat.log10(1000),mat.log10(1250),mat.log10(1600),mat.log10(2000),mat.log10(2500),mat.log10(3150),mat.log10(4000),mat.log10(5000),mat.log10(6300),mat.log10(8000),
        mat.log10(10000),mat.log10(12500),mat.log10(16000),mat.log10(20000)]
    
for st in sts:
    if st in stations:

        df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]]
        if len(df) > 0:


            fig = plt.figure(figsize=(11.7,8.3)) #A4
            #df_P = pd.DataFrame([])

            for p in Periods:
                cc = (df.index >=  p[0]) & (df.index < p[1])
                df1 = df.loc[cc]
                #df_P= pd.concat([df_P, df.loc[cc]])#df_P.append(df.loc[cc])

                mediana = []
                con = 0
                for f in ff:

                    mediana.append((df1[F[f]].median()))#-medianaToKeep[con])#(np.nanpercentile(spl,0.5))#
                    #print(df1[F[f]].median())
                    con += 1
                if p[2] == 'Year':    
                    plt.plot(xx,mediana,'ko-',linewidth=3.0,markersize=3,label=p[2])
                else:
                    plt.plot(xx,mediana,'o-',markersize=3,label=p[2])    

            xx1 = [mat.log10(80),mat.log10(125),mat.log10(200),mat.log10(315),mat.log10(500),mat.log10(800),
                    mat.log10(1250),mat.log10(2000),mat.log10(3150),mat.log10(5000),mat.log10(8000),
                    mat.log10(12500),mat.log10(20000)]
            xti = ('80','125','200','315','500','800','1250','2000','3150','5000','8000','12500','20000')

            if 'ylim' in locals():
                    if len(ylim) > 0:
                        plt.ylim(ylim)

            plt.xticks(xx1, xti)
            plt.xticks(fontsize=15)
            plt.yticks(fontsize=15)
            plt.grid(True,linestyle='dotted', linewidth=.5)
            plt.ylabel(lab, fontsize=15)
            plt.xlabel('1/3 OCTAVE FILTER CENTER FREQUENCIES [Hz]', fontsize=15)
            plt.title(STATIONS_SHORT[st], fontsize=15,fontweight = 'bold')
            plt.legend()
            plt.savefig(dirOutput + 'MedianPeriodCfr/' + STATIONS_SHORT[st] + "_" + FREQ[f] + '_median_period' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
            plt.show()
            plt.close()
            
        else:
            print('station', STATIONS[st], 'no data')
    else:
        print('station', STATIONS[st], 'not loaded')

if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim


[Back to graph list](#0) / [Change start and end time](#20)

<a id =14> </a> **Comparing stations**

<a id =G20> </a> **Stations percentiles plot**              
Output will be put in dirOutput/percStaz folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select x lim, leave blank for auto.                 
**input example:**
```python
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

xlim = [60,90] #x lim, [inf,sup] or [] for auto
```

In [ ]:
sts = []
freq = [63]

xlim = []

In [ ]:
if os.path.isdir(dirOutput + 'percStaz') == False:
    os.mkdir(dirOutput + 'percStaz')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
    
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
y = list(range(100,-1,-1))

for f in freq:
    if f in F:
        for st in sts:
            if st in stations:

                percen = []
                df = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]][F[f]]
                
                if len(df) > 0:
                    for i in range(0,101):
                        percen.append(df.quantile(i/100))

                    plt.plot((percen),y,label=STATIONS_SHORT[st])

                else:
                    print('station', STATIONS[st], 'no data')
            else:
                if st not in STATIONS_SHORT:
                    print('station #',st, 'not in Stations')
                else:
                    print('station', STATIONS[st], 'not loaded')

        plt.title(FREQ[f] + ' | ' + str(t_init)[0:10] + " - " + str(t_final)[0:10], fontsize=15,fontweight = 'bold')
        plt.xlabel(lab, fontsize=15)
        plt.ylabel('Exceedance Level', fontsize=15) 
        ax = plt.gca() #get axes
        if 'xlim' in locals():
            if len(xlim) > 0:
                ax.set_xlim(xlim)
                    

        xlim = ax.get_xlim()
        plt.plot([xlim[0],xlim[1]],[10,10],'--k',linewidth=.5)
        plt.plot([xlim[0],xlim[1]],[90,90],'--k',linewidth=.5)
        plt.plot([xlim[0],xlim[1]],[50,50],'--k',linewidth=.5) #,label = '10/50/75 prc'

        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)

        ax.set_xlim(xlim)
        plt.legend()
        plt.grid(False)
        plt.savefig(dirOutput + 'percStaz/' + FREQ[f] +'_percStaz' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
        plt.show()
        plt.close()
    else:
        print('Frequency ', f, ' not in Frequencies list')


if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G21> </a> **Boxplot of different stations**              
Output will be put in dirOutput/boxPlotStaz folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select periods to plot 
Select y lim, leave blank for auto.                 
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = []
freq = [125]

ylim = []

In [ ]:
if os.path.isdir(dirOutput + 'boxPlotStaz') == False:
    os.mkdir(dirOutput + 'boxPlotStaz')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
    
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90
ddf = []
for st in sts:
    if st in stations:
        ddf.append(merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]])
    else:
        if st not in STATIONS_SHORT:
            print('station #',st, 'not in Stations')
        else:
            print('station', STATIONS[st], 'not loaded')

merged_box2 = pd.concat(ddf)
            
for f in freq:
    if f in F:
        medianprops = dict(linestyle='--', linewidth=2.5, color='red',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')
        medianpointprops = dict(marker='+', markersize = 3,markeredgecolor='red',
                      markerfacecolor='red')


        meanpointprops = dict(marker='o', markersize = 3,markeredgecolor='black',
                      markerfacecolor='black')


        sns.set(style="whitegrid")
        ax = sns.boxplot(x = 'Station',y = F[f],color='w',linewidth=1.5,whis = (whi1,whi2),showmeans = False,meanprops = meanpointprops,
                medianprops = medianprops, showfliers = False,data = merged_box2)

            
        plt.xticks(fontsize=8)
        if 'ylim' in locals():
            if len(ylim) > 0:
                plt.ylim(ylim)
        plt.ylabel(lab,fontsize=15)
        plt.title(FREQ[f] + ' | ' + str(t_init)[0:10] + " - " + str(t_final)[0:10], fontsize=15,fontweight = 'bold')

        plt.grid(True)

        plt.savefig(dirOutput + 'boxPlotStaz/' + FREQ[f] +'_boxplotStaz' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
        plt.show()
        plt.close()
    else:
        print('Frequency ', f, ' not in Frequencies list')
    
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2

del merged_box2
del ddf

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G22> </a> **Violinplot of different stations**             
Output will be put in dirOutput/violinPlotStaz folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values of the whiskers (0 - 100, default 10 and .90).          
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select periods to plot 
Select y lim, leave blank for auto.                 
**input example:**
```python
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

ylim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
whi1 = 10 #first percentile (first whiskers value) 
whi2 = 90 #last percentile (second whiskers value)

sts = []
freq = [125]

ylim = [50,150]

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'violinPlotStaz') == False:
    os.mkdir(dirOutput + 'violinPlotStaz')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
    
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
if not 'whi1' in locals():
    whi1 = 10
if not 'whi2' in locals():
    whi2 = 90

ddf = []
for st in sts:
    if st in stations:
        ddf.append(merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]])
    else:
        if st not in STATIONS_SHORT:
            print('station #',st, 'not in Stations')
        else:
            print('station', STATIONS[st], 'not loaded')

merged_box2 = pd.concat(ddf)            

for f in freq:
    if f in F:
        
        sns.set(style="whitegrid")
        medianprops = dict(linestyle='', linewidth=1.5, color='k',label = 'median\n25-75prc\n' + str(whi1) + '-' + str(whi2) +'prc')

        sns.boxplot(x = 'Station',y = F[f],linewidth=1.5,whis = (whi1,whi2),showmeans = False,medianprops = medianprops,
            width=0.3, showfliers = False,showbox = False, data=merged_box2)

        sns.violinplot(x = 'Station', y = F[f],color='w', linewidth=1, inner = 'box',data=merged_box2)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        if 'ylim' in locals():
            if len(ylim) > 0:
                plt.ylim(ylim)
        plt.ylabel(lab, fontsize=15)
        plt.title(FREQ[f] + ' | ' + str(t_init)[0:10] + " - " + str(t_final)[0:10], fontsize=15,fontweight = 'bold')
        plt.grid(True)
        plt.savefig(dirOutput + 'violinPlotStaz/' + FREQ[f] +'_ViolinplotStaz' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
                    
        plt.show()
        plt.close()
    else:
        print('Frequency ', f, ' not in Frequencies list')
    
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'whi1' in locals():
    del whi1
if 'whi2' in locals():
    del whi2
del merged_box2

[Back to graph list](#0) / [Change start and end time](#20)

<a id =G23> </a> **Density plot for different stations**          
Output will be put in dirOutput/densPlotStaz folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select stations (leave it empty to use all loaded stations)                            
Select frequencies (leave it empty to use 63, 125, 250, 4000 Hz)
Select y lim, leave blank for auto.                 
**input example:**
```python
sts = [1,2,5]
freq = [4000] #Frequencies to plot,[F1,F2,..Fn] or [] for [63,125,250,4000]

xlim = [60,90] #y lim, [inf,sup] or [] for auto
```

In [ ]:
sts = [1,2]
freq = [125]

xlim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'densPlotStaz') == False:
    os.mkdir(dirOutput + 'densPlotStaz')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations
    
if 'freq' in locals(): 
    if len(freq) == 0:
        freq = [63,125,250,4000]
else:
    freq = [63,125,250,4000]
    
for f in freq:
    if f in F:
        for st in sts:
            if st in stations:
                ddf = merged_box.loc[merged_box['Station'] == STATIONS_SHORT[st]][F[f]]
                if len(ddf) > 0:
                    sns.kdeplot(ddf,label = STATIONS_SHORT[st] ) #y = df[F[f]]
                else:
                    print(STATIONS[st] + ' - ' + FREQ[f] + ' | ' + str(t_init)[0:10] + " - " + str(t_final)[0:10])
                    print('no data')
                    continue
            else:
                if st not in STATIONS_SHORT:
                    print('station #',st, 'not in Stations')
                else:
                    print('station', STATIONS[st], 'not loaded')
                continue
                
        if 'xlim' in locals():
            if len(xlim) > 0:
                ax.set_xlim(xlim)
                #g.set(xlim=xlim)
        plt.xlabel(lab, fontsize=15)    
        plt.ylabel('Probability')    
        plt.title(FREQ[f] + ' | ' + str(t_init)[0:10] + " - " + str(t_final)[0:10], fontsize=15,fontweight = 'bold')
        plt.legend()
        plt.grid(False)
        plt.savefig(dirOutput + 'densPlotStaz/' + FREQ[f] +'_densplotStaz' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
        plt.show()
        plt.close()
    else:
        print('Frequency ', f, ' not in Frequencies list')
    
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

[Back to graph list](#0) / [Change start and end time](#20)

**Plot of the median values of each frequencies, aggregated in all the stations in the selected period with percentiles band**     
Output will be put in dirOutput/medianFreq folder.

<span style='color:red'>*________________________________________________________________*                        
</span>*If left blank, default values will be used.*                 
Select the percentiles values to plot (0 - 1, default .1 and .9).                                     
Select y lim [inf,sup], leave blank for auto.              
**input example:**
```python
prc1 = .1 #first percentile (first whiskers value) 
prc2 = .9 #last percentile (second whiskers value)

ylim = [60,90] # [] for auto
```

In [ ]:
prc1 = 0.1 #first percentile (first whiskers value) 
prc2 = 0.9 #last percentile (second whiskers value)

ylim = []

In [ ]:
#check if output dir exists
if os.path.isdir(dirOutput + 'medianFreq') == False:
    os.mkdir(dirOutput + 'medianFreq')

if 'sts' in locals(): 
    if len(sts) == 0:
        sts = stations
else:
    sts = stations 

for st in stations:
    if st < 10:
        break
    
if not 'prc1' in locals():
    prc1 = 0.1
if not 'prc2' in locals():
    prc2 = 0.9
    

df = merged_box
if len(df) > 0:
    #List of frequencies
#    if secondi == 20:
#        ff = [63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,
#            10000,12500,16000,20000]
#    else:
    ff = [50,63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,
        10000,12500,16000,20000]
    mediana = []
    percenMin = []
    percenMax = []
    percen25 = []
    percen75 = []
    percen1 = []
    percen99 = []
    splTot = []
    splav = []
    mean = []

    fig = plt.figure(figsize=(11.7,8.3)) #A4
    
    for f in ff:

        mediana.append((df[F[f]].median()))#(np.nanpercentile(spl,0.5))#
        percenMin.append(df[F[f]].quantile(prc1))
        percenMax.append(df[F[f]].quantile(prc2))
        percen75.append(df[F[f]].quantile(0.75))
        percen25.append(df[F[f]].quantile(0.25))
        percen1.append(df[F[f]].quantile(0.99))
        percen99.append(df[F[f]].quantile(0.01))
        SPLT,SPL_av = totalSPL(df,f)
        mean.append(SPL_av)

        la = FREQ[f] + ': ' + str('{:.2f}')
        print(la.format(df[F[f]].quantile(0.99)))

        #SPLT,SPL_av = totalSPL(df,f)
        #splTot.append(SPLT)
        #splav.append(SPL_av) 

    medianaToKeep = mediana.copy()
    #if secondi == 60:
    xx = [mat.log10(50),mat.log10(63),mat.log10(80),
            mat.log10(100),mat.log10(125),mat.log10(160),mat.log10(200),mat.log10(250),mat.log10(315),mat.log10(400),mat.log10(500),mat.log10(630),mat.log10(800),
            mat.log10(1000),mat.log10(1250),mat.log10(1600),mat.log10(2000),mat.log10(2500),mat.log10(3150),mat.log10(4000),mat.log10(5000),mat.log10(6300),mat.log10(8000),
            mat.log10(10000),mat.log10(12500),mat.log10(16000),mat.log10(20000)]
#    else:
#        xx = [mat.log10(63),mat.log10(80),
#                mat.log10(100),mat.log10(125),mat.log10(160),mat.log10(200),mat.log10(250),mat.log10(315),mat.log10(400),mat.log10(500),mat.log10(630),mat.log10(800),
#                mat.log10(1000),mat.log10(1250),mat.log10(1600),mat.log10(2000),mat.log10(2500),mat.log10(3150),mat.log10(4000),mat.log10(5000),mat.log10(6300),mat.log10(8000),
#                mat.log10(10000),mat.log10(12500),mat.log10(16000),mat.log10(20000)]
    
    ax = plt.plot(xx,mediana,'bo-',markersize=3,label='median')
    ax = plt.fill_between(xx, percenMin, percenMax, color='r', alpha=.1,label=str(int(prc1*100)) + '-' + str(int(prc2*100)) + 'EL')
    ax = plt.fill_between(xx, percen25, percen75, color='g', alpha=.1,label=str(int(0.25*100)) + '-' + str(int(0.75*100)) + 'EL')
    ax = plt.plot(xx,percen1,'k--',markersize=3,label=' 1 EL')
    ax = plt.plot(xx,percen99,'k--',markersize=3,label='99 EL')

    ax = plt.gca() #get axes

    handles, labels = ax.get_legend_handles_labels()
    lgd = ax.legend(handles, labels, loc=1, bbox_to_anchor=(1.27, 1), fontsize=15)

#    if secondi == 60:
    xx1 = [mat.log10(50),mat.log10(63),mat.log10(80),mat.log10(125),mat.log10(200),mat.log10(315),mat.log10(500),mat.log10(800),
            mat.log10(1250),mat.log10(2000),mat.log10(3150),mat.log10(5000),mat.log10(8000),
            mat.log10(12500),mat.log10(20000)]
    xti = ('50','63','80','125','200','315','500','800','1250','2000','3150','5000','8000','12500','20000')
#    else:
#        xx1 = [mat.log10(63),mat.log10(80),mat.log10(125),mat.log10(200),mat.log10(315),mat.log10(500),mat.log10(800),
#                mat.log10(1250),mat.log10(2000),mat.log10(3150),mat.log10(5000),mat.log10(8000),
#                mat.log10(12500),mat.log10(20000)]
#        xti = ('63','80','125','200','315','500','800','1250','2000','3150','5000','8000','12500','20000')

    #plt.ylim(65,110)

    plt.xticks(xx1, xti)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.ylabel(lab,fontsize=15)
    plt.xlabel('1/3 OCTAVE FILTER CENTER FREQUENCIES [Hz]',fontsize=15)
    if 'ylim' in locals():
        if len(ylim) > 0:
            plt.ylim(ylim)
    plt.title(str(first_time[st])[0:10] + " - " + str(last_time[st])[0:10], fontsize=15,fontweight = 'bold')
    plt.legend()
    plt.savefig(dirOutput + 'medianFreq/TOT_Frequecnies_AVERAGED' + ext + '.png', dpi=300, facecolor='w', edgecolor='w')
    plt.show()
    plt.close()
    del ff
else:
    print('no data')

        
if 'ylim' in locals():
    del ylim
if 'xlim' in locals():
    del xlim

if 'prc1' in locals():
    del prc1
if 'prc2' in locals():
    del prc2

del st


[Back to graph list](#0) / [Change start and end time](#20)